In [21]:
"""
Скрипт для обработки тестов в порядке и составе выбранном пользователем.
"""

'\nСкрипт для обработки тестов в порядке и составе выбранном пользователем.\n'

In [22]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
import time
import re

In [23]:
file_data_xlsx_complex = 'data/complex.xlsx'
file_params = 'data/параметры ДЦОК ОПТЛ СППУ ДДО.xlsx'
path_to_end_folder_complex = 'data'

threshold_complex =7

In [24]:
def create_out_str_ddo(x):
    """
    Функция для создания выходной строки ДДО
    """
    return f'{dct_desciprion.get(x, "Проверьте правильность написания ответа в форме,в колонке ДДО_Обработанный_результат указаны несовпадающие значения")}\nРекомендуемые профессии:\n{dct_prof.get(x, "Проверьте правильность написания ответа в форме,в колонке ДДО_Обработанный_результат указаны несовпадающие значения")}'


def processing_finish_result_ddo(row):
    """
    Обработка результатов тестирования ДДО
    """
    # Создаем словарь для хранения данных
    dct_type = {'Человек-природа': 0, 'Человек-техника': 0, 'Человек-человек': 0, 'Человек-знаковые системы': 0,
                'Человек-художественный образ': 0}
    dct_error = {}  # словарь для хранения ошибочных  значений, для того чтобы было легче находить ошибки при обновлении
    # 1
    if row[0] == 'Ухаживать за животными.':
        dct_type['Человек-природа'] += 1
    elif row[0] == 'Обслуживать машины, приборы (следить, регулировать).':
        dct_type['Человек-техника'] += 1
    else:
        dct_error['Вопрос №1'] = f'Полученное значение-{row[0]} не совпадает с эталонными:[Ухаживать за животными.] или [Обслуживать машины, приборы (следить, регулировать).]'
    # 2
    if row[1] == 'Помогать больным.':
        dct_type['Человек-человек'] += 1
    elif row[1] == 'Составлять таблицы, схемы, компьютерные программы.':
        dct_type['Человек-знаковые системы'] += 1
    else:
        dct_error['Вопрос №2'] = f'Полученное значение-{row[1]} не совпадает с эталонными:[Помогать больным.] или [Составлять таблицы, схемы, компьютерные программы.]'

    # 3
    if row[2] == 'Следить за качеством книжных иллюстраций, плакатов, художественных открыток, музыкальных записей.':
        dct_type['Человек-художественный образ'] += 1
    elif row[2] == 'Следить за состоянием и развитием растений.':
        dct_type['Человек-природа'] += 1
    else:
        dct_error['Вопрос №3'] = f'Полученное значение-{row[2]} не совпадает с эталонными:[Следить за качеством книжных иллюстраций, плакатов, художественных открыток, музыкальных записей.] или [Следить за состоянием и развитием растений.]'

    # 4
    if row[3] == 'Обрабатывать материалы (дерево, ткань, металл, пластмассу и т.п.).':
        dct_type['Человек-техника'] += 1
    elif row[3] == 'Доводить товары до потребителя, рекламировать, продавать.':
        dct_type['Человек-человек'] += 1
    else:
        dct_error['Вопрос №4'] = f'Полученное значение-{row[3]} не совпадает с эталонными:[Обрабатывать материалы (дерево, ткань, металл, пластмассу и т.п.).] или [Доводить товары до потребителя, рекламировать, продавать.]'

    # 5
    if row[4] == 'Обсуждать научно-популярные книги, статьи.':
        dct_type['Человек-знаковые системы'] += 1
    elif row[4] == 'Обсуждать художественные книги (или пьесы, концерты).':
        dct_type['Человек-художественный образ'] += 1
    else:
        dct_error['Вопрос №5'] = f'Полученное значение-{row[4]} не совпадает с эталонными:[Обсуждать научно-популярные книги, статьи.] или [Обсуждать художественные книги (или пьесы, концерты).]'

    # 6
    if row[5] == 'Выращивать молодняк (животных какой-либо породы).':
        dct_type['Человек-природа'] += 1
    elif row[5] == 'Тренировать товарищей (или младших) для выполнения и закрепления каких-либо навыков (трудовых, учебных, спортивных).':
        dct_type['Человек-человек'] += 1
    else:
        dct_error['Вопрос №6'] = f'Полученное значение-{row[5]} не совпадает с эталонными:[Выращивать молодняк (животных какой-либо породы).] или [Тренировать товарищей (или младших) для выполнения и закрепления каких-либо навыков (трудовых, учебных, спортивных).]'

    # 7
    if row[6] == 'Копировать рисунки, изображения (или настраивать музыкальные инструменты).':
        dct_type['Человек-художественный образ'] += 1
    elif row[6] == 'Управлять какой-либо машиной (грузовым, подъемным или транспортным средством) - подъемным краном, трактором, тепловозом и др.':
        dct_type['Человек-техника'] += 1
    else:
        dct_error['Вопрос №7'] = f'Полученное значение-{row[6]} не совпадает с эталонными:[Копировать рисунки, изображения (или настраивать музыкальные инструменты).] или [Управлять какой-либо машиной (грузовым, подъемным или транспортным средством) - подъемным краном, трактором, тепловозом и др.]'

    # 8
    if row[7] == 'Сообщать, разъяснять людям нужные им сведения (в справочном бюро, на экскурсии и т.д.).':
        dct_type['Человек-человек'] += 1
    elif row[7] == 'Оформлять выставки, витрины (или участвовать в подготовке пьес, концертов).':
        dct_type['Человек-художественный образ'] += 1
    else:
        dct_error['Вопрос №8'] = f'Полученное значение-{row[7]} не совпадает с эталонными:[Сообщать, разъяснять людям нужные им сведения (в справочном бюро, на экскурсии и т.д.).] или [Оформлять выставки, витрины (или участвовать в подготовке пьес, концертов).]'

    # 9
    if row[8] == 'Ремонтировать вещи, изделия (одежду, технику), жилище.':
        dct_type['Человек-техника'] += 1
    elif row[8] == 'Искать и исправлять ошибки в текстах, таблицах, рисунках.':
        dct_type['Человек-знаковые системы'] += 1
    else:
        dct_error['Вопрос №9'] = f'Полученное значение-{row[8]} не совпадает с эталонными:[Ремонтировать вещи, изделия (одежду, технику), жилище.] или [Искать и исправлять ошибки в текстах, таблицах, рисунках.]'

    # 10
    if row[9] == 'Лечить животных.':
        dct_type['Человек-природа'] += 1
    elif row[9] == 'Выполнять вычисления, расчёты.':
        dct_type['Человек-знаковые системы'] += 1
    else:
        dct_error['Вопрос №10'] = f'Полученное значение-{row[9]} не совпадает с эталонными:[Лечить животных.] или [Выполнять вычисления, расчёты.]'

    # 11
    if row[10] == 'Выводить новые сорта растений.':
        dct_type['Человек-природа'] += 1
    elif row[10] == 'Конструировать, новые виды промышленных изделий (машины, одежду, дома, продукты питания и т.п.).':
        dct_type['Человек-техника'] += 1
    else:
        dct_error['Вопрос №11'] = f'Полученное значение-{row[10]} не совпадает с эталонными:[Выводить новые сорта растений.] или [Конструировать, новые виды промышленных изделий (машины, одежду, дома, продукты питания и т.п.).]'

    # 12
    if row[11] == 'Разбирать споры, ссоры между людьми, убеждать, разъяснять, наказывать, поощрять.':
        dct_type['Человек-человек'] += 1
    elif row[11] == 'Разбираться в чертежах, схемах, таблицах (проверять, уточнять, приводить в порядок).':
        dct_type['Человек-знаковые системы'] += 1
    else:
        dct_error['Вопрос №12'] = f'Полученное значение-{row[11]} не совпадает с эталонными:[Разбирать споры, ссоры между людьми, убеждать, разъяснять, наказывать, поощрять.] или [Разбираться в чертежах, схемах, таблицах (проверять, уточнять, приводить в порядок).]'

    # 13
    if row[12] == 'Наблюдать, изучать работу коллективов художественной самодеятельности.':
        dct_type['Человек-художественный образ'] += 1
    elif row[12] == 'Наблюдать, изучать жизнь микробов.':
        dct_type['Человек-природа'] += 1
    else:
        dct_error['Вопрос №13'] = f'Полученное значение-{row[12]} не совпадает с эталонными:[Наблюдать, изучать работу коллективов художественной самодеятельности.] или [Наблюдать, изучать жизнь микробов.]'

    # 14
    if row[13] == 'Обслуживать, налаживать медицинские приборы, аппараты.':
        dct_type['Человек-техника'] += 1
    elif row[13] == 'Оказывать людям медицинскую помощь при ранениях, ушибах, ожогах и т.п.':
        dct_type['Человек-человек'] += 1
    else:
        dct_error['Вопрос №14'] = f'Полученное значение-{row[13]} не совпадает с эталонными:[Обслуживать, налаживать медицинские приборы, аппараты.] или [Оказывать людям медицинскую помощь при ранениях, ушибах, ожогах и т.п.]'

    # 15
    if row[14] == 'Художественно описывать, изображать события (наблюдаемые и представляемые).':
        dct_type['Человек-знаковые системы'] += 1
    elif row[14] == 'Составлять точные описания-отчеты о наблюдаемых явлениях, событиях, измеряемых объектах и др.':
        dct_type['Человек-художественный образ'] += 1
    else:
        dct_error['Вопрос №15'] = f'Полученное значение-{row[14]} не совпадает с эталонными:[Художественно описывать, изображать события (наблюдаемые и представляемые).] или [Составлять точные описания-отчеты о наблюдаемых явлениях, событиях, измеряемых объектах и др.]'


    # 16
    if row[15] == 'Делать лабораторные анализы в больнице.':
        dct_type['Человек-природа'] += 1
    elif row[15] == 'Принимать, осматривать больных, беседовать с ними, назначать лечение.':
        dct_type['Человек-человек'] += 1
    else:
        dct_error['Вопрос №16'] = f'Полученное значение-{row[15]} не совпадает с эталонными:[Делать лабораторные анализы в больнице.] или [Принимать, осматривать больных, беседовать с ними, назначать лечение.]'

    # 17
    if row[16] == 'Красить или расписывать стены помещений, поверхность изделий.':
        dct_type['Человек-техника'] += 1
    elif row[16] == 'Осуществлять монтаж или сборку машин, приборов.':
        dct_type['Человек-техника'] += 1
    else:
        dct_error['Вопрос №17'] = f'Полученное значение-{row[16]} не совпадает с эталонными:[Красить или расписывать стены помещений, поверхность изделий.] или [Осуществлять монтаж или сборку машин, приборов.]'

    # 18
    if row[
        17] == 'Организовывать культпоходы сверстников или младших в театры, музеи, экскурсии, туристические походы и т.п.':
        dct_type['Человек-человек'] += 1
    elif row[17] == 'Играть на сцене, принимать участие в концертах.':
        dct_type['Человек-художественный образ'] += 1
    else:
        dct_error['Вопрос №18'] = f'Полученное значение-{row[17]} не совпадает с эталонными:[Организовывать культпоходы сверстников или младших в театры, музеи, экскурсии, туристические походы и т.п.] или [Играть на сцене, принимать участие в концертах.]'

    # 19
    if row[18] == 'Изготовлять по чертежам детали, изделия (машины, одежду), строить здания.':
        dct_type['Человек-техника'] += 1
    elif row[18] == 'Заниматься черчением, копировать чертежи, карты.':
        dct_type['Человек-знаковые системы'] += 1
    else:
        dct_error['Вопрос №19'] = f'Полученное значение-{row[18]} не совпадает с эталонными:[Изготовлять по чертежам детали, изделия (машины, одежду), строить здания.] или [Заниматься черчением, копировать чертежи, карты.]'

    # 20
    if row[19] == 'Вести борьбу с болезнями растений, с вредителями леса, сада.':
        dct_type['Человек-природа'] += 1
    elif row[19] == 'Работать на устройствах с клавиатурой, ноутбуке и др.).':
        dct_type['Человек-знаковые системы'] += 1
    else:
        dct_error['Вопрос №20'] = f'Полученное значение-{row[19]} не совпадает с эталонными:[Вести борьбу с болезнями растений, с вредителями леса, сада.] или [Работать на устройствах с клавиатурой, ноутбуке и др.).]'


    if len(dct_error) > 0:
        begin_str = 'Скопируйте правильные значения для указанных вопросов из квадратных скобок в вашу форму. \n'
        # создаем строку с результатами
        for sphere, value in dct_error.items():
            begin_str += f'{sphere} - {value};\n'
        return begin_str
    else:
        # возвращаем элемент с максимальным значением
        return max(dct_type, key=dct_type.get)

def processing_result_ddo(row):
    """
    Обработка результатов тестирования ДДО
    """
    # Создаем словарь для хранения данных
    # Создаем словарь для хранения данных
    dct_type = {'Человек-природа': 0, 'Человек-техника': 0, 'Человек-человек': 0, 'Человек-знаковые системы': 0,
                'Человек-художественный образ': 0}
    dct_error = {}  # словарь для хранения ошибочных  значений, для того чтобы было легче находить ошибки при обновлении
    # 1
    if row[0] == 'Ухаживать за животными.':
        dct_type['Человек-природа'] += 1
    elif row[0] == 'Обслуживать машины, приборы (следить, регулировать).':
        dct_type['Человек-техника'] += 1
    else:
        dct_error[
            'Вопрос №1'] = f'Полученное значение-{row[0]} не совпадает с эталонными:[Ухаживать за животными.] или [Обслуживать машины, приборы (следить, регулировать).]'
    # 2
    if row[1] == 'Помогать больным.':
        dct_type['Человек-человек'] += 1
    elif row[1] == 'Составлять таблицы, схемы, компьютерные программы.':
        dct_type['Человек-знаковые системы'] += 1
    else:
        dct_error[
            'Вопрос №2'] = f'Полученное значение-{row[1]} не совпадает с эталонными:[Помогать больным.] или [Составлять таблицы, схемы, компьютерные программы.]'

    # 3
    if row[2] == 'Следить за качеством книжных иллюстраций, плакатов, художественных открыток, музыкальных записей.':
        dct_type['Человек-художественный образ'] += 1
    elif row[2] == 'Следить за состоянием и развитием растений.':
        dct_type['Человек-природа'] += 1
    else:
        dct_error[
            'Вопрос №3'] = f'Полученное значение-{row[2]} не совпадает с эталонными:[Следить за качеством книжных иллюстраций, плакатов, художественных открыток, музыкальных записей.] или [Следить за состоянием и развитием растений.]'

    # 4
    if row[3] == 'Обрабатывать материалы (дерево, ткань, металл, пластмассу и т.п.).':
        dct_type['Человек-техника'] += 1
    elif row[3] == 'Доводить товары до потребителя, рекламировать, продавать.':
        dct_type['Человек-человек'] += 1
    else:
        dct_error[
            'Вопрос №4'] = f'Полученное значение-{row[3]} не совпадает с эталонными:[Обрабатывать материалы (дерево, ткань, металл, пластмассу и т.п.).] или [Доводить товары до потребителя, рекламировать, продавать.]'

    # 5
    if row[4] == 'Обсуждать научно-популярные книги, статьи.':
        dct_type['Человек-знаковые системы'] += 1
    elif row[4] == 'Обсуждать художественные книги (или пьесы, концерты).':
        dct_type['Человек-художественный образ'] += 1
    else:
        dct_error[
            'Вопрос №5'] = f'Полученное значение-{row[4]} не совпадает с эталонными:[Обсуждать научно-популярные книги, статьи.] или [Обсуждать художественные книги (или пьесы, концерты).]'

    # 6
    if row[5] == 'Выращивать молодняк (животных какой-либо породы).':
        dct_type['Человек-природа'] += 1
    elif row[
        5] == 'Тренировать товарищей (или младших) для выполнения и закрепления каких-либо навыков (трудовых, учебных, спортивных).':
        dct_type['Человек-человек'] += 1
    else:
        dct_error[
            'Вопрос №6'] = f'Полученное значение-{row[5]} не совпадает с эталонными:[Выращивать молодняк (животных какой-либо породы).] или [Тренировать товарищей (или младших) для выполнения и закрепления каких-либо навыков (трудовых, учебных, спортивных).]'

    # 7
    if row[6] == 'Копировать рисунки, изображения (или настраивать музыкальные инструменты).':
        dct_type['Человек-художественный образ'] += 1
    elif row[
        6] == 'Управлять какой-либо машиной (грузовым, подъемным или транспортным средством) - подъемным краном, трактором, тепловозом и др.':
        dct_type['Человек-техника'] += 1
    else:
        dct_error[
            'Вопрос №7'] = f'Полученное значение-{row[6]} не совпадает с эталонными:[Копировать рисунки, изображения (или настраивать музыкальные инструменты).] или [Управлять какой-либо машиной (грузовым, подъемным или транспортным средством) - подъемным краном, трактором, тепловозом и др.]'

    # 8
    if row[7] == 'Сообщать, разъяснять людям нужные им сведения (в справочном бюро, на экскурсии и т.д.).':
        dct_type['Человек-человек'] += 1
    elif row[7] == 'Оформлять выставки, витрины (или участвовать в подготовке пьес, концертов).':
        dct_type['Человек-художественный образ'] += 1
    else:
        dct_error[
            'Вопрос №8'] = f'Полученное значение-{row[7]} не совпадает с эталонными:[Сообщать, разъяснять людям нужные им сведения (в справочном бюро, на экскурсии и т.д.).] или [Оформлять выставки, витрины (или участвовать в подготовке пьес, концертов).]'

    # 9
    if row[8] == 'Ремонтировать вещи, изделия (одежду, технику), жилище.':
        dct_type['Человек-техника'] += 1
    elif row[8] == 'Искать и исправлять ошибки в текстах, таблицах, рисунках.':
        dct_type['Человек-знаковые системы'] += 1
    else:
        dct_error[
            'Вопрос №9'] = f'Полученное значение-{row[8]} не совпадает с эталонными:[Ремонтировать вещи, изделия (одежду, технику), жилище.] или [Искать и исправлять ошибки в текстах, таблицах, рисунках.]'

    # 10
    if row[9] == 'Лечить животных.':
        dct_type['Человек-природа'] += 1
    elif row[9] == 'Выполнять вычисления, расчёты.':
        dct_type['Человек-знаковые системы'] += 1
    else:
        dct_error[
            'Вопрос №10'] = f'Полученное значение-{row[9]} не совпадает с эталонными:[Лечить животных.] или [Выполнять вычисления, расчёты.]'

    # 11
    if row[10] == 'Выводить новые сорта растений.':
        dct_type['Человек-природа'] += 1
    elif row[10] == 'Конструировать, новые виды промышленных изделий (машины, одежду, дома, продукты питания и т.п.).':
        dct_type['Человек-техника'] += 1
    else:
        dct_error[
            'Вопрос №11'] = f'Полученное значение-{row[10]} не совпадает с эталонными:[Выводить новые сорта растений.] или [Конструировать, новые виды промышленных изделий (машины, одежду, дома, продукты питания и т.п.).]'

    # 12
    if row[11] == 'Разбирать споры, ссоры между людьми, убеждать, разъяснять, наказывать, поощрять.':
        dct_type['Человек-человек'] += 1
    elif row[11] == 'Разбираться в чертежах, схемах, таблицах (проверять, уточнять, приводить в порядок).':
        dct_type['Человек-знаковые системы'] += 1
    else:
        dct_error[
            'Вопрос №12'] = f'Полученное значение-{row[11]} не совпадает с эталонными:[Разбирать споры, ссоры между людьми, убеждать, разъяснять, наказывать, поощрять.] или [Разбираться в чертежах, схемах, таблицах (проверять, уточнять, приводить в порядок).]'

    # 13
    if row[12] == 'Наблюдать, изучать работу коллективов художественной самодеятельности.':
        dct_type['Человек-художественный образ'] += 1
    elif row[12] == 'Наблюдать, изучать жизнь микробов.':
        dct_type['Человек-природа'] += 1
    else:
        dct_error[
            'Вопрос №13'] = f'Полученное значение-{row[12]} не совпадает с эталонными:[Наблюдать, изучать работу коллективов художественной самодеятельности.] или [Наблюдать, изучать жизнь микробов.]'

    # 14
    if row[13] == 'Обслуживать, налаживать медицинские приборы, аппараты.':
        dct_type['Человек-техника'] += 1
    elif row[13] == 'Оказывать людям медицинскую помощь при ранениях, ушибах, ожогах и т.п.':
        dct_type['Человек-человек'] += 1
    else:
        dct_error[
            'Вопрос №14'] = f'Полученное значение-{row[13]} не совпадает с эталонными:[Обслуживать, налаживать медицинские приборы, аппараты.] или [Оказывать людям медицинскую помощь при ранениях, ушибах, ожогах и т.п.]'

    # 15
    if row[14] == 'Художественно описывать, изображать события (наблюдаемые и представляемые).':
        dct_type['Человек-знаковые системы'] += 1
    elif row[14] == 'Составлять точные описания-отчеты о наблюдаемых явлениях, событиях, измеряемых объектах и др.':
        dct_type['Человек-художественный образ'] += 1
    else:
        dct_error[
            'Вопрос №15'] = f'Полученное значение-{row[14]} не совпадает с эталонными:[Художественно описывать, изображать события (наблюдаемые и представляемые).] или [Составлять точные описания-отчеты о наблюдаемых явлениях, событиях, измеряемых объектах и др.]'

    # 16
    if row[15] == 'Делать лабораторные анализы в больнице.':
        dct_type['Человек-природа'] += 1
    elif row[15] == 'Принимать, осматривать больных, беседовать с ними, назначать лечение.':
        dct_type['Человек-человек'] += 1
    else:
        dct_error[
            'Вопрос №16'] = f'Полученное значение-{row[15]} не совпадает с эталонными:[Делать лабораторные анализы в больнице.] или [Принимать, осматривать больных, беседовать с ними, назначать лечение.]'

    # 17
    if row[16] == 'Красить или расписывать стены помещений, поверхность изделий.':
        dct_type['Человек-техника'] += 1
    elif row[16] == 'Осуществлять монтаж или сборку машин, приборов.':
        dct_type['Человек-техника'] += 1
    else:
        dct_error[
            'Вопрос №17'] = f'Полученное значение-{row[16]} не совпадает с эталонными:[Красить или расписывать стены помещений, поверхность изделий.] или [Осуществлять монтаж или сборку машин, приборов.]'

    # 18
    if row[
        17] == 'Организовывать культпоходы сверстников или младших в театры, музеи, экскурсии, туристические походы и т.п.':
        dct_type['Человек-человек'] += 1
    elif row[17] == 'Играть на сцене, принимать участие в концертах.':
        dct_type['Человек-художественный образ'] += 1
    else:
        dct_error['Вопрос №18'] = f'Полученное значение-{row[17]} не совпадает с эталонными:[Организовывать культпоходы сверстников или младших в театры, музеи, экскурсии, туристические походы и т.п.] или [Играть на сцене, принимать участие в концертах.]'

    # 19
    if row[18] == 'Изготовлять по чертежам детали, изделия (машины, одежду), строить здания.':
        dct_type['Человек-техника'] += 1
    elif row[18] == 'Заниматься черчением, копировать чертежи, карты.':
        dct_type['Человек-знаковые системы'] += 1
    else:
        dct_error[
            'Вопрос №19'] = f'Полученное значение-{row[18]} не совпадает с эталонными:[Изготовлять по чертежам детали, изделия (машины, одежду), строить здания.] или [Заниматься черчением, копировать чертежи, карты.]'

    # 20
    if row[19] == 'Вести борьбу с болезнями растений, с вредителями леса, сада.':
        dct_type['Человек-природа'] += 1
    elif row[19] == 'Работать на устройствах с клавиатурой, ноутбуке и др.).':
        dct_type['Человек-знаковые системы'] += 1
    else:
        dct_error[
            'Вопрос №20'] = f'Полученное значение-{row[19]} не совпадает с эталонными:[Вести борьбу с болезнями растений, с вредителями леса, сада.] или [Работать на устройствах с клавиатурой, ноутбуке и др.).]'

    if len(dct_error) > 0:
        begin_str = 'Скопируйте правильные значения для указанных вопросов из квадратных скобок в вашу форму. \n'
        # создаем строку с результатами
        for sphere, value in dct_error.items():
            begin_str += f'{sphere} - {value};\n'
        return begin_str
    else:
        # сортируем по убыванию
        result_lst = sorted(dct_type.items(), key=lambda t: t[1], reverse=True)
        begin_str = ''
        # создаем строку с результатами
        for sphere, value in result_lst:
            begin_str += f'{sphere} - {value};\n'

        return begin_str

In [25]:
def processing_result_sppu(row):
    """
    Обработка результатов тестирования
    """

    # Создаем словарь для хранения данных
    dct_type = {'сфера работы с людьми': 0, 'сфера умственного труда': 0, 'сфера технических интересов': 0,
                'сфера эстетики и искусства': 0,
                'сфера физического труда, подвижной деятельности': 0,
                'сфера материальных интересов, планово-экономических видов работ': 0}
    dct_error = {}  # словарь для хранения ошибочных  значений, для того чтобы было легче находить ошибки при обновлении

    # 1
    if row[0] == 'общаться с самыми разными людьми;':
        dct_type['сфера работы с людьми'] += 1
    elif row[0] == 'что-нибудь делать своими руками – мебель, одежду, машины и т.д.;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[0] == 'снимать фильмы, рисовать, писать книги, выступать на сцене и т.д.':
        dct_type['сфера эстетики и искусства'] += 1
    else:
        dct_error['Вопрос №1'] = f'Полученное значение-{row[0]} не совпадает с эталонными:[общаться с самыми разными людьми;], [что-нибудь делать своими руками – мебель, одежду, машины и т.д.;], [снимать фильмы, рисовать, писать книги, выступать на сцене и т.д.]'

    # 2
    if row[1] == 'художественная форма, мастерство писателя или режиссера;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[1] == 'сюжет, действие героев;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[1] == 'информация, которая может пригодиться в жизни.':
        dct_type['сфера умственного труда'] += 1
    else:
        dct_error['Вопрос №2'] = f'Полученное значение-{row[1]} не совпадает с эталонными:[художественная форма, мастерство писателя или режиссера;], [сюжет, действие героев;], [информация, которая может пригодиться в жизни.]'

    # 3
    if row[2] == 'в области науки;':
        dct_type['сфера умственного труда'] += 1
    elif row[2] == 'за общественную деятельность;':
        dct_type['сфера работы с людьми'] += 1
    elif row[2] == 'в области искусства.':
        dct_type['сфера эстетики и искусства'] += 1
    else:
        dct_error['Вопрос №3'] = f'Полученное значение-{row[2]} не совпадает с эталонными:[в области науки;], [за общественную деятельность;], [в области искусства.]'

    # 4
    if row[3] == 'управляющим банка;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[3] == 'главным инженером на производстве;':
        dct_type['сфера технических интересов'] += 1
    elif row[3] == 'начальником экспедиции.':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    else:
        dct_error['Вопрос №4'] = f'Полученное значение-{row[3]} не совпадает с эталонными:[управляющим банка;], [главным инженером на производстве;], [начальником экспедиции.]'

    # 5
    if row[4] == 'достижения науки;':
        dct_type['сфера умственного труда'] += 1
    elif row[4] == 'развитие производства;':
        dct_type['сфера технических интересов'] += 1
    elif row[4] == 'взаимопонимание среди людей.':
        dct_type['сфера работы с людьми'] += 1
    else:
        dct_error['Вопрос №5'] = f'Полученное значение-{row[4]} не совпадает с эталонными:[достижения науки;], [развитие производства;], [взаимопонимание среди людей.]'

    # 6
    if row[5] == 'благоустройством школы (столовая, спортзал, компьютеры);':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[5] == 'созданием дружного, сплоченного коллектива;':
        dct_type['сфера работы с людьми'] += 1
    elif row[5] == 'разработкой новых технологий обучения.':
        dct_type['сфера умственного труда'] += 1
    else:
        dct_error['Вопрос №6'] = f'Полученное значение-{row[5]} не совпадает с эталонными:[благоустройством школы (столовая, спортзал, компьютеры);], [созданием дружного, сплоченного коллектива;], [разработкой новых технологий обучения.]'

    # 7
    if row[6] == 'внешний вид экспонатов (цвет, форма);':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[6] == 'внутреннее устройство экспонатов (механизм);':
        dct_type['сфера умственного труда'] += 1
    elif row[6] == 'практическое применение экспонатов.':
        dct_type['сфера технических интересов'] += 1
    else:
        dct_error['Вопрос №7'] = f'Полученное значение-{row[6]} не совпадает с эталонными:[внешний вид экспонатов (цвет, форма);], [внутреннее устройство экспонатов (механизм);], [практическое применение экспонатов.]'

    # 8
    if row[7] == 'мужество, смелость, выносливость;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[7] == 'дружелюбие, чуткость, отзывчивость;':
        dct_type['сфера работы с людьми'] += 1
    elif row[7] == 'ответственность, аккуратность.':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    else:
        dct_error['Вопрос №8'] = f'Полученное значение-{row[7]} не совпадает с эталонными:[мужество, смелость, выносливость;], [дружелюбие, чуткость, отзывчивость;], [ответственность, аккуратность.]'

    # 9
    if row[8] == 'писать стихи или музыку или рисовать;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[8] == 'ставить различные опыты;':
        dct_type['сфера умственного труда'] += 1
    elif row[8] == 'тренироваться.':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    else:
        dct_error['Вопрос №9'] = f'Полученное значение-{row[8]} не совпадает с эталонными:[писать стихи или музыку или рисовать;], [ставить различные опыты;], [тренироваться.]'

    # 10
    if row[9] == 'экстремальный туризм (альпинизм, виндсерфинг, горные лыжи);':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[9] == 'деловое общение;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[9] == 'возможность знакомства с историей и культурой другой страны.':
        dct_type['сфера эстетики и искусства'] += 1
    else:
        dct_error['Вопрос №10'] = f'Полученное значение-{row[9]} не совпадает с эталонными:[экстремальный туризм (альпинизм, виндсерфинг, горные лыжи);], [деловое общение;], [возможность знакомства с историей и культурой другой страны.]'

    # 11
    if row[10] == 'о машине нового типа;':
        dct_type['сфера технических интересов'] += 1
    elif row[10] == 'о новой научной теории;':
        dct_type['сфера умственного труда'] += 1
    elif row[10] == 'о человеческих взаимоотношениях.':
        dct_type['сфера работы с людьми'] += 1
    else:
        dct_error['Вопрос №11'] = f'Полученное значение-{row[10]} не совпадает с эталонными:[о машине нового типа;], [о новой научной теории;], [о человеческих взаимоотношениях.]'

    # 12
    if row[11] == 'технический;':
        dct_type['сфера технических интересов'] += 1
    elif row[11] == 'музыкальный;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[11] == 'спортивный.':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    else:
        dct_error['Вопрос №12'] = f'Полученное значение-{row[11]} не совпадает с эталонными:[технический;], [музыкальный;], [спортивный.]'

    # 13
    if row[12] == 'улучшению взаимопонимания между учителями и учениками;':
        dct_type['сфера работы с людьми'] += 1
    elif row[12] == 'поддержанию здоровья учащихся, занятиям спортом;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[12] == 'укреплению дисциплины.':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    else:
        dct_error['Вопрос №13'] = f'Полученное значение-{row[12]} не совпадает с эталонными:[улучшению взаимопонимания между учителями и учениками;], [поддержанию здоровья учащихся, занятиям спортом;], [укреплению дисциплины.]'

    # 14
    if row[13] == 'научно-популярные фильмы;':
        dct_type['сфера умственного труда'] += 1
    elif row[13] == 'программы о культуре и спорте;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[13] == 'спортивные программы.':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    else:
        dct_error['Вопрос №14'] = f'Полученное значение-{row[13]} не совпадает с эталонными:[научно-популярные фильмы;], [программы о культуре и спорте;], [спортивные программы.]'

    # 15
    if row[14] == 'с машинами, механизмами;':
        dct_type['сфера технических интересов'] += 1
    elif row[14] == 'с объектами природы;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[14] == 'с детьми или сверстниками.':
        dct_type['сфера работы с людьми'] += 1
    else:
        dct_error['Вопрос №15'] = f'Полученное значение-{row[14]} не совпадает с эталонными:[с машинами, механизмами;], [с объектами природы;], [с детьми или сверстниками.]'

    # 16
    if row[15] == 'давать знания и умения;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[15] == 'учить общению с другими людьми;':
        dct_type['сфера работы с людьми'] += 1
    elif row[15] == 'обучать навыкам работы.':
        dct_type['сфера технических интересов'] += 1
    else:
        dct_error['Вопрос №16'] = f'Полученное значение-{row[15]} не совпадает с эталонными:[давать знания и умения;], [учить общению с другими людьми;], [обучать навыкам работы.]'

    # 17
    if row[16] == 'вести здоровый образ жизни;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[16] == 'иметь возможность заниматься творчеством;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[16] == 'иметь удобные бытовые условия.':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    else:
        dct_error['Вопрос №17'] = f'Полученное значение-{row[16]} не совпадает с эталонными:[вести здоровый образ жизни;], [иметь возможность заниматься творчеством;], [иметь удобные бытовые условия.]'

    # 18
    if row[17] == 'защита интересов и прав граждан;':
        dct_type['сфера работы с людьми'] += 1
    elif row[17] == 'забота о материальном благополучии людей;':
        dct_type['сфера технических интересов'] += 1
    elif row[17] == 'наука и технический прогресс.':
        dct_type['сфера умственного труда'] += 1
    else:
        dct_error['Вопрос №18'] = f'Полученное значение-{row[17]} не совпадает с эталонными:[защита интересов и прав граждан;], [забота о материальном благополучии людей;], [наука и технический прогресс.]'

    # 19
    if row[18] == 'физкультуры;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[18] == 'математики;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[18] == 'труда.':
        dct_type['сфера технических интересов'] += 1
    else:
        dct_error['Вопрос №19'] = f'Полученное значение-{row[18]} не совпадает с эталонными:[физкультуры;], [математики;], [труда.]'

    # 20
    if row[19] == 'планировать производство продукции;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[19] == 'изготавливать изделия;':
        dct_type['сфера технических интересов'] += 1
    elif row[19] == 'заниматься сбытом продукции.':
        dct_type['сфера работы с людьми'] += 1
    else:
        dct_error['Вопрос №20'] = f'Полученное значение-{row[19]} не совпадает с эталонными:[планировать производство продукции;], [изготавливать изделия;], [заниматься сбытом продукции.]'

    # 21
    if row[20] == 'о выдающихся ученых и их открытиях;':
        dct_type['сфера умственного труда'] += 1
    elif row[20] == 'о творчестве ученых и музыкантов;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[20] == 'об интересных изобретениях.':
        dct_type['сфера технических интересов'] += 1
    else:
        dct_error['Вопрос №21'] = f'Полученное значение-{row[20]} не совпадает с эталонными:[о выдающихся ученых и их открытиях;], [о творчестве ученых и музыкантов;], [об интересных изобретениях.]'

    # 22
    if row[21] == 'делая что-то по хозяйству;':
        dct_type['сфера технических интересов'] += 1
    elif row[21] == 'с книгой;':
        dct_type['сфера умственного труда'] += 1
    elif row[21] == 'на выставках, концертах и пр.':
        dct_type['сфера эстетики и искусства'] += 1
    else:
        dct_error['Вопрос №22'] = f'Полученное значение-{row[21]} не совпадает с эталонными:[делая что-то по хозяйству;], [с книгой;], [на выставках, концертах и пр.]'

    # 23
    if row[22] == 'о художественной выставке;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[22] == 'о ситуации на фондовой бирже;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[22] == 'о научном открытии.':
        dct_type['сфера умственного труда'] += 1
    else:
        dct_error['Вопрос №23'] = f'Полученное значение-{row[22]} не совпадает с эталонными:[о художественной выставке;], [о ситуации на фондовой бирже;], [о научном открытии.]'

    # 24
    if row[23] == 'в помещении, где много людей;':
        dct_type['сфера работы с людьми'] += 1
    elif row[23] == 'в необычных условиях;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[23] == 'в обычном кабинете.':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    else:
        dct_error['Вопрос №24'] = f'Полученное значение-{row[23]} не совпадает с эталонными:[в помещении, где много людей;], [в необычных условиях;], [в обычном кабинете.]'

    # проверяем на ошибки
    if len(dct_error) > 0:
        begin_str = 'Скопируйте правильные значения для указанных вопросов из квадратных скобок в вашу форму. \n'
        # создаем строку с результатами
        for sphere, value in dct_error.items():
            begin_str += f'{sphere} - {value};\n'
        return begin_str
    else:
        # сортируем по убыванию
        result_lst = sorted(dct_type.items() , key=lambda t : t[1] , reverse=True)
        begin_str = ''
        # создаем строку с результатами
        for sphere,value in result_lst:
            begin_str += f'{sphere} - {value};\n'


        #добавляем описание
        return begin_str

In [26]:
def processing_result_optl(row):
    """
    Функция для подсчета результатов теста на определение профессионального типа личности
    :return:
    """
    # Создаем словарь для хранения данных
    dct_type = {'Реалистический': 0, 'Интеллектуальный': 0, 'Социальный': 0,
                'Офисный': 0,
                'Предпринимательский': 0,
                'Артистический': 0}
    dct_error = {}  # словарь для хранения ошибочных  значений, для того чтобы было легче находить ошибки при обновлении
    # 1
    if row[0] == 'Автомеханик':
        dct_type['Реалистический'] += 1
    elif row[0] == 'Физиотерапевт':
        dct_type['Социальный'] += 1
    else:
        dct_error['Вопрос №1'] = f'Полученное значение-{row[0]} не совпадает с эталонными:[Автомеханик] или [Физиотерапевт]'

    # 2
    if row[1] == 'Специалист по защите информации':
        dct_type['Интеллектуальный'] += 1
    elif row[1] == 'Логистик':
        dct_type['Предпринимательский'] += 1
    else:
        dct_error['Вопрос №2'] = f'Полученное значение-{row[1]} не совпадает с эталонными:[Специалист по защите информации] или [Логистик]'

    # 3
    if row[2] == 'Оператор связи':
        dct_type['Офисный'] += 1
    elif row[2] == 'Кинооператор':
        dct_type['Артистический'] += 1
    else:
        dct_error['Вопрос №3'] = f'Полученное значение-{row[2]} не совпадает с эталонными:[Оператор связи] или [Кинооператор]'

    # 4
    if row[3] == 'Водитель':
        dct_type['Реалистический'] += 1
    elif row[3] == 'Продавец':
        dct_type['Социальный'] += 1
    else:
        dct_error['Вопрос №4'] = f'Полученное значение-{row[3]} не совпадает с эталонными:[Водитель] или [Продавец]'

    # 5
    if row[4] == 'Инженер-конструктор':
        dct_type['Интеллектуальный'] += 1
    elif row[4] == 'Менеджер по продажам':
        dct_type['Предпринимательский'] += 1
    else:
        dct_error['Вопрос №5'] = f'Полученное значение-{row[4]} не совпадает с эталонными:[Инженер-конструктор] или [Менеджер по продажам]'

    # 6
    if row[5] == 'Диспетчер':
        dct_type['Офисный'] += 1
    elif row[5] == 'Дизайнер компьютерных программ':
        dct_type['Артистический'] += 1
    else:
        dct_error['Вопрос №6'] = f'Полученное значение-{row[5]} не совпадает с эталонными:[Диспетчер] или [Дизайнер компьютерных программ]'

    # 7
    if row[6] == 'Ветеринар':
        dct_type['Реалистический'] += 1
    elif row[6] == 'Эколог':
        dct_type['Социальный'] += 1
    else:
        dct_error['Вопрос №7'] = f'Полученное значение-{row[6]} не совпадает с эталонными:[Ветеринар] или [Эколог]'

    # 8
    if row[7] == 'Биолог-исследователь':
        dct_type['Интеллектуальный'] += 1
    elif row[7] == 'Фермер':
        dct_type['Предпринимательский'] += 1
    else:
        dct_error['Вопрос №8'] = f'Полученное значение-{row[7]} не совпадает с эталонными:[Биолог-исследователь] или [Фермер]'

    # 9
    if row[8] == 'Лаборант':
        dct_type['Офисный'] += 1
    elif row[8] == 'Дрессировщик':
        dct_type['Артистический'] += 1
    else:
        dct_error['Вопрос №9'] = f'Полученное значение-{row[8]} не совпадает с эталонными:[Лаборант] или [Дрессировщик]'

    # 10
    if row[9] == 'Агроном':
        dct_type['Реалистический'] += 1
    elif row[9] == 'Санитарный врач':
        dct_type['Социальный'] += 1
    else:
        dct_error['Вопрос №10'] = f'Полученное значение-{row[9]} не совпадает с эталонными:[Агроном] или [Санитарный врач]'

    # 11
    if row[10] == 'Селекционер':
        dct_type['Интеллектуальный'] += 1
    elif row[10] == 'Заготовитель сельхозпродуктов':
        dct_type['Предпринимательский'] += 1
    else:
        dct_error['Вопрос №11'] = f'Полученное значение-{row[10]} не совпадает с эталонными:[Селекционер] или [Заготовитель сельхозпродуктов]'

    # 12
    if row[11] == 'Микробиолог':
        dct_type['Офисный'] += 1
    elif row[11] == 'Ландшафтный дизайнер':
        dct_type['Артистический'] += 1
    else:
        dct_error['Вопрос №12'] = f'Полученное значение-{row[11]} не совпадает с эталонными:[Микробиолог] или [Ландшафтный дизайнер]'

    # 13
    if row[12] == 'Массажист':
        dct_type['Реалистический'] += 1
    elif row[12] == 'Воспитатель':
        dct_type['Социальный'] += 1
    else:
        dct_error['Вопрос №13'] = f'Полученное значение-{row[12]} не совпадает с эталонными:[Массажист] или [Воспитатель]'

    # 14
    if row[13] == 'Преподаватель':
        dct_type['Интеллектуальный'] += 1
    elif row[13] == 'Предприниматель':
        dct_type['Предпринимательский'] += 1
    else:
        dct_error['Вопрос №14'] = f'Полученное значение-{row[13]} не совпадает с эталонными:[Преподаватель] или [Предприниматель]'

    # 15
    if row[14] == 'Администратор':
        dct_type['Офисный'] += 1
    elif row[14] == 'Режиссер театра и кино':
        dct_type['Артистический'] += 1
    else:
        dct_error['Вопрос №15'] = f'Полученное значение-{row[14]} не совпадает с эталонными:[Администратор] или [Режиссер театра и кино]'


    # 16
    if row[15] == 'Официант':
        dct_type['Реалистический'] += 1
    elif row[15] == 'Врач':
        dct_type['Социальный'] += 1
    else:
        dct_error['Вопрос №16'] = f'Полученное значение-{row[15]} не совпадает с эталонными:[Официант] или [Врач]'

    # 17
    if row[16] == 'Психолог':
        dct_type['Интеллектуальный'] += 1
    elif row[16] == 'Торговый агент':
        dct_type['Предпринимательский'] += 1
    else:
        dct_error['Вопрос №17'] = f'Полученное значение-{row[16]} не совпадает с эталонными:[Психолог] или [Торговый агент]'

    # 18
    if row[17] == 'Страховой агент':
        dct_type['Офисный'] += 1
    elif row[17] == 'Хореограф':
        dct_type['Артистический'] += 1
    else:
        dct_error['Вопрос №18'] = f'Полученное значение-{row[17]} не совпадает с эталонными:[Страховой агент] или [Хореограф]'

    # 19
    if row[18] == 'Ювелир-гравер':
        dct_type['Реалистический'] += 1
    elif row[18] == 'Журналист':
        dct_type['Социальный'] += 1
    else:
        dct_error['Вопрос №19'] = f'Полученное значение-{row[18]} не совпадает с эталонными:[Ювелир-гравер] или [Журналист]'

    # 20
    if row[19] == 'Искусствовед':
        dct_type['Интеллектуальный'] += 1
    elif row[19] == 'Продюсер':
        dct_type['Предпринимательский'] += 1
    else:
        dct_error['Вопрос №20'] = f'Полученное значение-{row[19]} не совпадает с эталонными:[Искусствовед] или [Продюсер]'

    # 21
    if row[20] == 'Редактор':
        dct_type['Офисный'] += 1
    elif row[20] == 'Музыкант':
        dct_type['Артистический'] += 1
    else:
        dct_error['Вопрос №21'] = f'Полученное значение-{row[20]} не совпадает с эталонными:[Редактор] или [Музыкант]'

    # 22
    if row[21] == 'Дизайнер интерьера':
        dct_type['Реалистический'] += 1
    elif row[21] == 'Экскурсовод':
        dct_type['Социальный'] += 1
    else:
        dct_error['Вопрос №22'] = f'Полученное значение-{row[21]} не совпадает с эталонными:[Дизайнер интерьера] или [Экскурсовод]'

    # 23
    if row[22] == 'Композитор':
        dct_type['Интеллектуальный'] += 1
    elif row[22] == 'Арт-директор':
        dct_type['Предпринимательский'] += 1
    else:
        dct_error['Вопрос №23'] = f'Полученное значение-{row[22]} не совпадает с эталонными:[Композитор] или [Арт-директор]'

    # 24
    if row[23] == 'Музейный работник':
        dct_type['Офисный'] += 1
    elif row[23] == 'Актер театра и кино':
        dct_type['Артистический'] += 1
    else:
        dct_error['Вопрос №24'] = f'Полученное значение-{row[23]} не совпадает с эталонными:[Музейный работник] или [Актер театра и кино]'

    # 25
    if row[24] == 'Верстальщик':
        dct_type['Реалистический'] += 1
    elif row[24] == 'Гид-переводчик':
        dct_type['Социальный'] += 1
    else:
        dct_error['Вопрос №25'] = f'Полученное значение-{row[24]} не совпадает с эталонными:[Верстальщик] или [Гид-переводчик]'

    # 26
    if row[25] == 'Лингвист':
        dct_type['Интеллектуальный'] += 1
    elif row[25] == 'Антикризисный управляющий':
        dct_type['Предпринимательский'] += 1
    else:
        dct_error['Вопрос №26'] = f'Полученное значение-{row[25]} не совпадает с эталонными:[Лингвист] или [Антикризисный управляющий]'

    # 27
    if row[26] == 'Корректор':
        dct_type['Офисный'] += 1
    elif row[26] == 'Художественный редактор':
        dct_type['Артистический'] += 1
    else:
        dct_error['Вопрос №27'] = f'Полученное значение-{row[26]} не совпадает с эталонными:[Корректор] или [Художественный редактор]'

    # 28
    if row[27] == 'Наборщик текстов':
        dct_type['Реалистический'] += 1
    elif row[27] == 'Юрисконсульт':
        dct_type['Социальный'] += 1
    else:
        dct_error['Вопрос №28'] = f'Полученное значение-{row[27]} не совпадает с эталонными:[Наборщик текстов] или [Юрисконсульт]'

    # 29
    if row[28] == 'Программист':
        dct_type['Интеллектуальный'] += 1
    elif row[28] == 'Брокер':
        dct_type['Предпринимательский'] += 1
    else:
        dct_error['Вопрос №29'] = f'Полученное значение-{row[28]} не совпадает с эталонными:[Программист] или [Брокер]'

    # 30
    if row[29] == 'Бухгалтер':
        dct_type['Офисный'] += 1
    elif row[29] == 'Литературный переводчик':
        dct_type['Артистический'] += 1
    else:
        dct_error['Вопрос №30'] = f'Полученное значение-{row[29]} не совпадает с эталонными:[Бухгалтер] или [Литературный переводчик]'

        # проверяем на ошибки
    if len(dct_error) > 0:
        begin_str = 'Скопируйте правильные значения для указанных вопросов из квадратных скобок в вашу форму. \n'
        # создаем строку с результатами
        for sphere, value in dct_error.items():
            begin_str += f'{sphere} - {value};\n'
        return begin_str
    else:
        # сортируем по убыванию
        result_lst = sorted(dct_type.items(), key=lambda t: t[1], reverse=True)
        begin_str = ''
        # создаем строку с результатами
        for sphere, value in result_lst:
            begin_str += f'{sphere} - {value};\n'

        return begin_str

In [27]:
def processing_result_dcok(row):
    """
    Функция для обработки результатов теста Диагностика ценностных ориентаций в карьере
    """
    # Создаем словарь для хранения данных
    dct_type = {'Профессиональная компетентность': 0, 'Менеджмент': 0, 'Автономия (независимость)': 0,
                'Стабильность работы': 0,
                'Стабильность места жительства': 0, 'Служение': 0, 'Вызов': 0,
                'Интеграция стилей жизни': 0, 'Предпринимательство': 0}
    dct_error = {}  # словарь для хранения ошибочных  значений, для того чтобы было легче находить ошибки при обновлении
    # 1
    dct_type['Профессиональная компетентность'] += row[0]

    # 2
    dct_type['Менеджмент'] += row[1]

    # 3
    dct_type['Автономия (независимость)'] += row[2]

    # 4
    dct_type['Стабильность работы'] += row[3]

    # 5
    dct_type['Служение'] += row[4]

    # 6
    dct_type['Вызов'] += row[5]

    # 7
    dct_type['Интеграция стилей жизни'] += row[6]

    # 8
    dct_type['Предпринимательство'] += row[7]

    # 9
    dct_type['Профессиональная компетентность'] += row[8]

    # 10
    dct_type['Менеджмент'] += row[9]

    # 11
    dct_type['Автономия (независимость)'] += row[10]

    # 12
    dct_type['Стабильность работы'] += row[11]

    # 13
    dct_type['Служение'] += row[12]

    # 14
    dct_type['Вызов'] += row[13]

    # 15
    dct_type['Интеграция стилей жизни'] += row[14]

    # 16
    dct_type['Предпринимательство'] += row[15]

    # 17
    dct_type['Профессиональная компетентность'] += row[16]

    # 18
    dct_type['Менеджмент'] += row[17]

    # 19
    dct_type['Автономия (независимость)'] += row[18]

    # 20
    dct_type['Стабильность места жительства'] += row[19]

    # 21
    dct_type['Служение'] += row[20]

    # 22
    dct_type['Вызов'] += row[21]

    # 23
    dct_type['Интеграция стилей жизни'] += row[22]

    # 24
    dct_type['Предпринимательство'] += row[23]

    # 25
    dct_type['Профессиональная компетентность'] += row[24]

    # 26
    dct_type['Менеджмент'] += row[25]

    # 27
    dct_type['Автономия (независимость)'] += row[26]

    # 28
    dct_type['Стабильность места жительства'] += row[27]

    # 29
    dct_type['Служение'] += row[28]

    # 30
    dct_type['Вызов'] += row[29]

    # 31
    dct_type['Интеграция стилей жизни'] += row[30]

    # 32
    dct_type['Предпринимательство'] += row[31]

    # 33
    dct_type['Профессиональная компетентность'] += row[32]

    # 34
    dct_type['Менеджмент'] += row[33]

    # 35
    dct_type['Автономия (независимость)'] += row[34]

    # 36
    dct_type['Стабильность работы'] += row[35]

    # 37
    dct_type['Служение'] += row[36]

    # 38
    dct_type['Вызов'] += row[37]

    # 39
    dct_type['Интеграция стилей жизни'] += row[38]

    # 40
    dct_type['Предпринимательство'] += row[39]

    # 41
    dct_type['Стабильность места жительства'] += row[40]

    # Сортируем словарь
    result_lst = sorted(dct_type.items(), key=lambda t: t[1], reverse=True)
    begin_str = ''
    # создаем строку с результатами
    for sphere, value in result_lst:
        begin_str += f'{sphere} - {value};\n'

    return begin_str

def processing_finish_result_dcok(row):
    """
Функция для вычисления итогового балла  результатов теста Диагностика ценностных ориентаций в карьере
"""

    # Создаем словарь для хранения данных
    dct_type = {'Профессиональная компетентность': 0, 'Менеджмент': 0, 'Автономия (независимость)': 0,
                'Стабильность работы': 0,
                'Стабильность места жительства': 0, 'Служение': 0, 'Вызов': 0,
                'Интеграция стилей жизни': 0, 'Предпринимательство': 0}
    dct_error = {}  # словарь для хранения ошибочных  значений, для того чтобы было легче находить ошибки при обновлении
    # 1
    dct_type['Профессиональная компетентность'] += row[0]

    # 2
    dct_type['Менеджмент'] += row[1]

    # 3
    dct_type['Автономия (независимость)'] += row[2]

    # 4
    dct_type['Стабильность работы'] += row[3]

    # 5
    dct_type['Служение'] += row[4]

    # 6
    dct_type['Вызов'] += row[5]

    # 7
    dct_type['Интеграция стилей жизни'] += row[6]

    # 8
    dct_type['Предпринимательство'] += row[7]

    # 9
    dct_type['Профессиональная компетентность'] += row[8]

    # 10
    dct_type['Менеджмент'] += row[9]

    # 11
    dct_type['Автономия (независимость)'] += row[10]

    # 12
    dct_type['Стабильность работы'] += row[11]

    # 13
    dct_type['Служение'] += row[12]

    # 14
    dct_type['Вызов'] += row[13]

    # 15
    dct_type['Интеграция стилей жизни'] += row[14]

    # 16
    dct_type['Предпринимательство'] += row[15]

    # 17
    dct_type['Профессиональная компетентность'] += row[16]

    # 18
    dct_type['Менеджмент'] += row[17]

    # 19
    dct_type['Автономия (независимость)'] += row[18]

    # 20
    dct_type['Стабильность места жительства'] += row[19]

    # 21
    dct_type['Служение'] += row[20]

    # 22
    dct_type['Вызов'] += row[21]

    # 23
    dct_type['Интеграция стилей жизни'] += row[22]

    # 24
    dct_type['Предпринимательство'] += row[23]

    # 25
    dct_type['Профессиональная компетентность'] += row[24]

    # 26
    dct_type['Менеджмент'] += row[25]

    # 27
    dct_type['Автономия (независимость)'] += row[26]

    # 28
    dct_type['Стабильность места жительства'] += row[27]

    # 29
    dct_type['Служение'] += row[28]

    # 30
    dct_type['Вызов'] += row[29]

    # 31
    dct_type['Интеграция стилей жизни'] += row[30]

    # 32
    dct_type['Предпринимательство'] += row[31]

    # 33
    dct_type['Профессиональная компетентность'] += row[32]

    # 34
    dct_type['Менеджмент'] += row[33]

    # 35
    dct_type['Автономия (независимость)'] += row[34]

    # 36
    dct_type['Стабильность работы'] += row[35]

    # 37
    dct_type['Служение'] += row[36]

    # 38
    dct_type['Вызов'] += row[37]

    # 39
    dct_type['Интеграция стилей жизни'] += row[38]

    # 40
    dct_type['Предпринимательство'] += row[39]

    # 41
    dct_type['Стабильность места жительства'] += row[40]

    # Делим на 5 результаты
    for key, value in dct_type.items():
        dct_type[key] = round(dct_type[key] / 5)
    # Сортируем
    result_lst = sorted(dct_type.items(), key=lambda t: t[1], reverse=True)

    begin_str = ''
    # создаем строку с результатами
    for sphere, value in result_lst:
        begin_str += f'{sphere} - {value};\n'

    return begin_str

In [28]:
def processing_dcok():
    """
    Фугкция для обработки данных ДЦОК
    :return:
    """
    try:
        threshold_dcok = var_entry_threshold_dcok.get()
        df = pd.read_excel(file_data_xlsx_dcok)
        if df.shape[1] - threshold_dcok <= 41:
            raise WrongNumberColumn

        base_df = df.iloc[:, :threshold_dcok]  # создаем датафрейм с данными не относящимися к тесту
        # делаем строковыми названия колонок
        base_df.columns = list(map(str, base_df.columns))

        # заменяем пробелы на нижнее подчеркивание и очищаем от пробельных символов в начале и конце
        base_df.columns = [column.strip().replace(' ', '_') for column in base_df.columns]

        # очищаем от всех символов кроме букв цифр
        base_df.columns = [re.sub(r'[^_\d\w]', '', column) for column in base_df.columns]

        answers_df = df.iloc[:, threshold_dcok:threshold_dcok + 41]  # датафрейм с результатами

        answers_df.columns = [f'ДЦОК_Вопрос_ №_{i}' for i in range(1, answers_df.shape[1] + 1)]

        answers_df = answers_df.astype(int)

        # Создаем колонку для результов первичного подсчета
        answers_df['ДЦОК_Необработанный_результат'] = answers_df.apply(processing_result_dcok, axis=1)
        answers_df['ДЦОК_Необработанный_результат'] = answers_df['ДЦОК_Необработанный_результат'].astype(str)

        # Создаем колонку для итоговой обработки
        answers_df['ДЦОК_Обработанный_результат'] = answers_df.apply(processing_finish_result_dcok, axis=1)
        answers_df['ДЦОК_Обработанный_результат'] = answers_df['ДЦОК_Обработанный_результат'].astype(str)

        # соединяем после обработки
        df = pd.concat([base_df, answers_df], axis=1)
        description_result = """
По каждой из девяти карьерных ориентаций подсчитывается количество баллов. Таким образом определяется ведущая карьерная ориентация - количество набранных для этого баллов должно быть не менее пяти. Иногда ведущей не становится ни одна карьерная ориентация - в таком случае карьера не является центральной в жизни личности. 

1) Профессиональная компетентность -быть профессионалом, мастером в своем деле. 
Эта ориентация связана с наличием способностей и талантов в определенной области.
Люди с такой ориентацией хотят быть мастерами своего дела, они бывают особенно счастливы, когда достигают успеха в профессиональной сфере, но быстро теряют интерес к работе, которая не позволяет развивать их способности. Вряд ли их заинтересует даже значительно более высокая должность, если она не связана с их профессиональными компетенциями. Они ищут признания своих талантов, что должно выражаться в статусе, соответствующем их мастерству. Они готовы управлять другими в пределах своей компетенции, но управление не представляет для них особого интереса. Поэтому многие из этой категории отвергают работу руководителя, управление рассматривают как необходимое условие для продвижения в своей профессиональной сфере.
2) Менеджмент - Управлять – людьми, проектами, бизнес-процессами и т.п.
 Для этих людей первостепенное значение имеет ориентация личности на интеграцию усилий других людей, полнота ответственности за конечный результат и соединение различных функций организации. С возрастом и опытом эта карьерная ориентация проявляется сильнее. Возможности для лидерства, высокого дохода, повышенных уровней ответственности и вклад в успех своей организации являются ключевыми ценностями и мотивами. Самое главное для них – управление: людьми, проектами, любыми бизнес-процессами – это в целом не имеет принципиального значения. Центральное понятие их профессионального развития – власть, осознание того, что от них зависит принятие ключевых решений. Причем для них не является принципиальным управление собственным проектом или целым бизнесом, скорее наоборот, они в большей степени ориентированы на построение карьеры в наемном менеджменте, но при условии, что им будут делегированы значительные полномочия. Человек с такой ориентацией будет считать, что не достиг цели своей карьеры, пока не займет должность, на которой будет управлять различными сторонами деятельности предприятия.
3) Автономия (независимость) – Главное в работе – это свобода и независимость. 
Первичная забота личности с такой ориентацией –освобождение от организационных правил, предписаний и ограничений. Они испытывают трудности, связанные с установленными правилами, процедурами, рабочим днем, дисциплиной, формой одежды и т.д. Они любят выполнять работу своим способом, темпом и по собственным стандартам. Они не любят, когда работа вмешивается в их частную жизнь, поэтому предпочитают делать независимую карьеру собственным путем. Они скорее выберут низкосортную работу, чем откажутся от автономии и независимости. Для них первоочередная задача развития карьеры – получить возможность работать самостоятельно, самому решать, как, когда и что делать для достижения тех или иных целей. Карьера для них – это, прежде всего, способ реализации их свободы, поэтому любые рамки и строгое подчинение оттолкнут их даже от внешне привлекательной вакансии. Такой человек может работать в организации, которая обеспечивает достаточную степень свободы.
4) Стабильность работы - стабильная, надежная работа на длительное время.
 Эти люди испытывают потребность в безопасности, защите и возможности прогнозирования и будут искать постоянную работу с минимальной вероятностью увольнения. Эти люди отождествляют свою работу со своей карьерой. Их потребность в безопасности и стабильности ограничивает выбор вариантов	карьеры.
Авантюрные или краткосрочные проекты и только становящиеся на ноги компании их, скорее всего, не привлекают. Они очень ценят социальные гарантии, которые может предложить работодатель, и, как правило, их выбор места работы связан именно с длительным контрактом и стабильным положением компании на рынке. Такие люди ответственность за управление своей карьерой перекладывают на нанимателя. Часто данная ценностная ориентация сочетается с невысоким уровнем притязаний.
5) Стабильность места жительства - Главное – жить в своем городе (минимум переездов, командировок).
 Важнее остаться на одном месте жительства, чем получить повышение или новую работу на новой местности. Переезд для таких людей неприемлем, и даже частые командировки являются для них негативным фактором при рассмотрении	предложения о работе.
6) Служение - Воплощать в работе свои идеалы и ценности.
 Данная ценностная ориентация характерна для людей, занимающихся делом по причине желания реализовать в своей работе главные ценности. Они часто ориентированы больше на ценности, чем на требующиеся в данном виде работы способности. Они стремятся приносить пользу людям, обществу, для них очень важно видеть конкретные плоды своей работы, даже если они и не выражены в материальном эквиваленте. Основной тезис построения их карьеры – получить возможность максимально эффективно использовать их таланты и опыт для реализации общественно	важной цели. Люди, ориентированные на служение, общительны и часто консервативны. Человек с такой ориентацией не будет работать в организации, которая враждебна его целям и ценностям.
7) Вызов - Сделать   невозможное – возможным, решать   уникальные   задачи. 
Эти люди считают успехом преодоление непреодолимых препятствий, решение неразрешимых проблем или просто выигрыш. Они ориентированы на то, чтобы “бросать вызов”. Для одних людей вызов представляет более трудная работа, для других это — конкуренция и межличностные отношения. Они ориентированы на решение заведомо сложных задач, преодоление препятствий ради победы в конкурентной борьбе. Они чувствуют себя преуспевающими только тогда, когда постоянно вовлечены в решение трудных проблем или в ситуацию соревнования. Карьера для них – это постоянный вызов их профессионализму, и они всегда готовы его принять. Социальная ситуация чаще всего рассматривается с позиции “выигрыша – проигрыша”. Процесс борьбы и победа более важна для них, чем конкретная область деятельности или квалификация. Новизна, разнообразие и вызов имеют для них очень большую ценность, и, если все идет слишком просто, им становиться скучно.
8) Интеграция стилей жизни - Сохранение гармонии между сложившейся личной жизнью и карьерой.
 Для людей этой категории карьера должна ассоциироваться с общим стилем жизни, уравновешивая потребности человека, семьи и карьеры. Они хотят, чтобы организационные отношения отражали бы уважение к их личным и семейным проблемам.
Выбирать и поддерживать определенный образ жизни для них важнее, чем добиваться успеха в карьере. Развитие карьеры их привлекает только в том случае, если она не нарушает привычный им стиль жизни и окружение. Для них важно, чтобы все было уравновешено – карьера, семья, личные интересы и т.п. Жертвовать   чем-то   одним   ради   другого   им    явно    не    свойственно. Такие люди обычно в своем поведении проявляют конформность (тенденция изменять свое поведение в зависимости от влияния других людей с тем, чтобы оно соответствовало мнению окружающих)
9) Предпринимательство – Создавать новые организации, товары, услуги.
 Этим людям нравится создавать новые организации, товары или услуги, которые могут быть отождествлены с их усилиями. Работать на других – это не их, они – предприниматели по духу, и цель их карьеры – создать что-то новое, организовать свое дело, воплотить в жизнь идею, всецело принадлежащую только им. Вершина карьеры в их понимании – собственный бизнес.
"""

        if 'ФИО' in df.columns:
            df['ДЦОК_Описание_результата'] = df['ФИО'] + '.' + ' \nДиагностика ценностных ориентаций в карьере.\n' + df[
                'ДЦОК_Обработанный_результат'] + description_result
        else:
            df['ДЦОК_Описание_результата'] = 'Диагностика ценностных ориентаций в карьере.\n' + df[
                'ДЦОК_Обработанный_результат'] + description_result

        # генерируем текущее время
        t = time.localtime()
        current_time = time.strftime('%H_%M_%S', t)

        df.to_excel(f'{path_to_end_folder_dcok}/Полная таблица с результатами ДЦОК от {current_time}.xlsx', index=False,
                    engine='xlsxwriter')

        # Создаем сокращенный вариант
        send_df = df.iloc[:, :threshold_dcok]
        # Добавляем колонки с результатами
        send_df['ДЦОК_Необработанный_результат'] = df['ДЦОК_Необработанный_результат']
        send_df['ДЦОК_Обработанный_результат'] = df['ДЦОК_Обработанный_результат']
        send_df['ДЦОК_Описание_результата'] = df['ДЦОК_Описание_результата']
        send_df.to_excel(f'{path_to_end_folder_dcok}/Краткая таблица с результатами ДЦОК  от {current_time}.xlsx',
                         index=False, engine='xlsxwriter')


    except NameError:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'Выберите файлы с данными и папку куда будет генерироваться файл')
    except KeyError as e:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'В таблице не найдена указанная колонка {e.args}')
    except FileNotFoundError:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'Перенесите файлы которые вы хотите обработать в корень диска. Проблема может быть\n '
                             f'в слишком длинном пути к обрабатываемым файлам')
    except WrongNumberColumn:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'Неправильное количество колонок в таблице!\n'
                             f'Проверьте количество вопросов в тестах!\n'
                             f'ДЦОК -41 колонка т.е.41 тестовый вопрос\n'
                             f'ОПТЛ - 30 колонок т.е. 30 тестовых вопросов\n'
                             f'СППУ - 24 колонки т.е. 24 тестовых вопроса\n'
                             f'ДДО - 20 колонок т.е. 20 тестовых вопросов')
    else:
        messagebox.showinfo('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                            'Данные успешно обработаны')

In [29]:
def processing_optl():
    """
    Функция для обработки результатов на определение профессионального типа личности
    """
    try:
        threshold_optl = var_entry_threshold_optl.get()

        df = pd.read_excel(file_data_xlsx_optl)

        if df.shape[1] - threshold_optl <= 30:
            raise WrongNumberColumn

        base_df = df.iloc[:, :threshold_optl]  # создаем датафрейм с данными не относящимися к тесту
        # делаем строковыми названия колонок
        base_df.columns = list(map(str, base_df.columns))

        # заменяем пробелы на нижнее подчеркивание и очищаем от пробельных символов в начале и конце
        base_df.columns = [column.strip().replace(' ', '_') for column in base_df.columns]

        # очищаем от всех символов кроме букв цифр
        base_df.columns = [re.sub(r'[^_\d\w]', '', column) for column in base_df.columns]

        answers_df = df.iloc[:, threshold_optl:threshold_optl+30]  # датафрейм с результатами

        answers_df.columns = [f'ОТПЛ_Вопрос_ №_{i}' for i in range(1, answers_df.shape[1] + 1)]

        answers_df = answers_df.astype(str)

        answers_df = answers_df.applymap(lambda x: x.strip())
        answers_df['ОПТЛ_Обработанный_результат'] = answers_df.apply(processing_result_optl, axis=1)
        answers_df['ОПТЛ_Обработанный_результат'] = answers_df['ОПТЛ_Обработанный_результат'].astype(str)

        # соединяем после обработки
        df = pd.concat([base_df, answers_df], axis=1)

        description_result = """
Обработка результатов теста 
8-10 баллов – ярко выраженный тип;  
5-7 баллов – средне выраженный тип;  
2-4 баллов – слабо выраженный тип.  
Наибольшее количество баллов указывает на доминирующий тип. В чистом виде эти профессиональные типы встречаются редко – обычно можно говорить только о преобладающем типе личности. Выбирая профессию, необходимо учитывать свой профессиональный тип. Если профессия не соответствует вашему типу личности, работа будет даваться вам ценой значительного нервно-психического напряжения.  
Интерпретация результатов теста 
1. Реалистический тип (Р) 
Люди, относящиеся к этому типу, предпочитают выполнять работу, требующую силы, ловкости, подвижности, хорошей координации движений, навыков практической работы. Результаты труда профессионалов этого типа ощутимы и реальны – их руками создан весь окружающий нас предметный мир. Люди реалистического типа охотнее делают, чем говорят, они настойчивы и уверены в себе, в работе предпочитают четкие и конкретные указания. Придерживаются традиционных ценностей, поэтому критически относятся к новым идеям.  
Близкие типы: интеллектуальный и офисный.  
Противоположный тип: социальный.  
Хороший продавец и хороший ремонтник никогда не будут голодать. Шенк 
2. Интеллектуальный (И) 
Людей, относящихся к этому типу, отличают аналитические способности, рационализм, независимость и оригинальность мышления, умение точно формулировать и излагать свои мысли, решать логические задачи, генерировать новые идеи. Они часто выбирают научную и исследовательскую работу. Им нужна свобода для творчества. Работа способна увлечь их настолько, что стирается грань между рабочим временем и досугом. Мир идей для них может быть важнее, чем общение с людьми. Материальное благополучие для них обычно не на первом месте.  
Близкие типы: реалистический и артистический.  
Противоположный тип: предпринимательский.  
Научная работа не подходит человеку, который обеими ногами стоит на земле и обеими руками тянется к долларам. М.Ларни 
3. Социальный (С) 
Люди, относящиеся к этому типу, предпочитают профессиональную деятельность, связанную с обучением, воспитанием, лечением, консультированием, обслуживанием. Люди этого типа гуманны, чувствительны, активны, ориентированы на социальные нормы, способны понять эмоциональное состояние другого человека. Для них характерно хорошее речевое развитие, живая мимика, интерес к людям, готовность прийти на помощь. Материальное благополучие для них обычно не на первом месте.  
Близкие типы: артистический и предпринимательский.  
Противоположный тип: реалистический.  
Если больному после разговора с врачом не стало легче, то это не врач. В.Бехтерев 
4. Офисный (О) 
Люди этого типа обычно проявляют склонность к работе, связанной с обработкой и систематизацией информации, предоставленной в виде условных знаков, цифр, формул, текстов (ведение документации, установление количественных соотношений между числами и условными знаками). Они отличаются аккуратностью, пунктуальностью, практичностью, ориентированы на социальные нормы, предпочитают четко регламентированную работу. Материальное благополучие для них более значимо, чем для других типов. Склонны к работе, не связанной с широкими контактами и принятием ответственных решений.  
Близкие типы: реалистический и предпринимательский.  
Противоположный тип: артистический.  
Офис может работать без шефа, но не без секретаря. Дж.Фонда 
5. Предпринимательский (П)  
Люди этого типа находчивы, практичны, быстро ориентируются в сложной обстановке, склонны к самостоятельному принятию решений, социально активны, готовы рисковать, ищут острые ощущения. Любят и умеют общаться. Имеют высокий уровень притязаний. Избегают занятий, требующих усидчивости, большой и длительной концентрации внимания. Для них значимо материальное благополучие. Предпочитают деятельность, требующую энергии, организаторских способностей, связанную с руководством, управлением и влиянием на людей.  
Близкие типы: офисный и социальный.  
Противоположный тип: исследовательский.  
Специальность налетчика куда менее заманчива, чем смежные с ней профессии политика или биржевого спекулянта. О.Генри  
6. Артистический (А)  
Люди этого типа оригинальны, независимы в принятии решений, редко ориентируются на социальные нормы и одобрение, обладают необычным взглядом на жизнь, гибкостью мышления, эмоциональной чувствительностью.
Отношения с людьми строят, опираясь на свои ощущения, эмоции, воображение, интуицию. Они не выносят жесткой регламентации, предпочитая свободный график работы. Часто выбирают профессии, связанные с литературой, театром, кино, музыкой, изобразительным искусством.  
Близкие типы: интеллектуальный и социальный.  
Противоположный тип: офисный.  
Только поэты и женщины умеют обращаться с деньгами так, как деньги того заслуживают. А.Боннар 
"""
        if 'ФИО' in df.columns:
            df['ОПТЛ_Описание_результата'] = df['ФИО'] + '.' + ' \nОпределение профессионального типа личности.\n' + df[
                'ОПТЛ_Обработанный_результат'] + description_result
        else:
            df['ОПТЛ_Описание_результата'] = 'Определение профессионального типа личности.\n' + df[
                'ОПТЛ_Обработанный_результат'] + description_result

        # генерируем текущее время
        t = time.localtime()
        current_time = time.strftime('%H_%M_%S', t)

        df.to_excel(f'{path_to_end_folder_optl}/Полная таблица с результатами ОПТЛ от {current_time}.xlsx', index=False,
                    engine='xlsxwriter')

        # Создаем сокращенный вариант
        send_df = df.iloc[:, :threshold_optl]
        # Добавляем колонки с результатами
        send_df['ОПТЛ_Обработанный_результат'] = df['ОПТЛ_Обработанный_результат']
        send_df['ОПТЛ_Описание_результата'] = df['ОПТЛ_Описание_результата']

        send_df.to_excel(f'{path_to_end_folder_optl}/Краткая таблица с результатами ОТПЛ  от {current_time}.xlsx',
                         index=False, engine='xlsxwriter')



    except NameError:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'Выберите файлы с данными и папку куда будет генерироваться файл')
    except KeyError as e:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'В таблице не найдена указанная колонка {e.args}')
    except FileNotFoundError:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'Перенесите файлы которые вы хотите обработать в корень диска. Проблема может быть\n '
                             f'в слишком длинном пути к обрабатываемым файлам')
    except WrongNumberColumn:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'Неправильное количество колонок в таблице!\n'
                             f'Проверьте количество вопросов в тестах!\n'
                             f'ДЦОК -41 колонка т.е.41 тестовый вопрос\n'
                             f'ОПТЛ - 30 колонок т.е. 30 тестовых вопросов\n'
                             f'СППУ - 24 колонки т.е. 24 тестовых вопроса\n'
                             f'ДДО - 20 колонок т.е. 20 тестовых вопросов')
    else:
        messagebox.showinfo('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                            'Данные успешно обработаны')

In [30]:
def processing_sppu():
    """
    Функция для обработки результатов тестирования сферы профессиональных предпочтений учащихся
    :return:
    """
    try:
        df = pd.read_excel(file_data_xlsx_sppu)
        threshold_sppu = var_entry_threshold_sppu.get()  # количество колонок не относящихся к вопросам

        if df.shape[1] - threshold_sppu <= 24:
            raise WrongNumberColumn

        base_df = df.iloc[:, :threshold_sppu]  # создаем датафрейм с данными не относящимися к тесту

        # делаем строковыми названия колонок
        base_df.columns = list(map(str, base_df.columns))
        # заменяем пробелы на нижнее подчеркивание и очищаем от пробельных символов в начале и конце
        base_df.columns = [column.strip().replace(' ', '_') for column in base_df.columns]

        # очищаем от всех символов кроме букв цифр
        base_df.columns = [re.sub(r'[^_\d\w]', '', column) for column in base_df.columns]

        answers_df = df.iloc[:, threshold_sppu:threshold_sppu+24]  # датафрейм с результатами

        answers_df.columns = [f'СППУ_Вопрос_ №_{i}' for i in range(1, answers_df.shape[1] + 1)]

        answers_df = answers_df.astype(str)

        answers_df = answers_df.applymap(lambda x: x.strip())

        answers_df['СППУ_Обработанный_результат'] = answers_df.apply(processing_result_sppu, axis=1)

        answers_df['СППУ_Обработанный_результат'] = answers_df['СППУ_Обработанный_результат'].astype(str)

        # соединяем после обработки
        df = pd.concat([base_df, answers_df], axis=1)

        # Создаем строку с описанием
        description_result = """
Шкала оценки результатов
10-12 баллов – ярко выраженная профессиональная склонность;
7-9 баллов – средне выраженная профессиональная склонность;
4-6 баллов – слабо выраженная профессиональная склонность;
0-3 баллов – профессиональная склонность не выражена.

Примеры профессий для каждой сферы
Сфера работы с людьми: учитель, педагог, экскурсовод, воспитатель,  социолог, психолог, менеджер по персоналу, следователь.
Сфера умственного труда: ученый-исследователь (математик, физик, химик, кибернетик, археолог, геолог),  инженер, юрист, врач, эколог, архитектор, продюсер.
Сфера технических интересов:  программист, электротехник, радиотехник, Web-мастер, статистик, водитель, технолог, диспетчер, секретарь-машинистка, телефонист.
Сфера эстетики и искусства: художник, дизайнер, писатель, поэт,  режиссер, артист, конструктор, косметолог, костюмер, гример, кондитер, портной-кутюрье, цветовод. 
Сфера физического труда,  подвижной деятельности: спортсмен, фотограф, экспедитор, парикмахер,  бармен, официант, стюардесса, продавец, закройщик, специалист по ремонту, кассир, медперсонал, бригадир, кладовщик,  почтальон,  фермер,  водитель-дальнобойщик, полицейский, военный.
Сфера материальных интересов,  планово-экономических видов работ: экономист, администратор, менеджер, предприниматель, аудитор, специалист по рекламе, брокер, агент страховых компаний, коммерсант, завхоз.
"""

        if 'ФИО' in df.columns:
            df['СППУ_Описание_результата'] = df['ФИО'] + '.' + ' \nСфера профессиональных предпочтений учащихся.\n' + \
                                             df['СППУ_Обработанный_результат'] + description_result
        else:
            df['СППУ_Описание_результата'] = 'Сфера профессиональных предпочтений учащихся.\n' + df[
                'СППУ_Обработанный_результат'] + description_result

        # генерируем текущее время
        t = time.localtime()
        current_time = time.strftime('%H_%M_%S', t)

        df.to_excel(f'{path_to_end_folder_sppu}/Полная таблица с результатами СППУ от {current_time}.xlsx', index=False,
                    engine='xlsxwriter')

        # Создаем сокращенный вариант
        send_df = df.iloc[:, :threshold_sppu]
        # Добавляем колонки с результатами
        send_df['СППУ_Обработанный_результат'] = df['СППУ_Обработанный_результат']
        send_df['СППУ_Описание_результата'] = df['СППУ_Описание_результата']
        send_df.to_excel(f'{path_to_end_folder_sppu}/Краткая таблица с результатами СППУ  от {current_time}.xlsx',
                         index=False, engine='xlsxwriter')

    except NameError:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'Выберите файлы с данными и папку куда будет генерироваться файл')
    except KeyError as e:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'В таблице не найдена указанная колонка {e.args}')
    except FileNotFoundError:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'Перенесите файлы которые вы хотите обработать в корень диска. Проблема может быть\n '
                             f'в слишком длинном пути к обрабатываемым файлам')
    except WrongNumberColumn:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'Неправильное количество колонок в таблице!\n'
                             f'Проверьте количество вопросов в тестах!\n'
                             f'ДЦОК -41 колонка т.е.41 тестовый вопрос\n'
                             f'ОПТЛ - 30 колонок т.е. 30 тестовых вопросов\n'
                             f'СППУ - 24 колонки т.е. 24 тестовых вопроса\n'
                             f'ДДО - 20 колонок т.е. 20 тестовых вопросов')
    else:
        messagebox.showinfo('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                            'Данные успешно обработаны')

In [31]:
def processing_ddo():
    """
    Фугкция для обработки данных ДДО
    :return:
    """
    try:


        # Создаем словари для создания текста письма
        global dct_prof
        dct_prof = {
            'Человек-природа': 'Тракторист, рыбовод, зоотехник, агроном, садовник, ветеринар, животновод, геолог, биолог, почвовед и т.д.',
            'Человек-техника': 'Водитель, токарь, инженер, слесарь, радиотехник, швея, электрик, механик, монтажник и т.п.',
            'Человек-человек': 'Продавец, медсестра, секретарь, бортпроводник, учитель, воспитатель, няня, преподаватель, врач, официант, администратор и т.п.',
            'Человек-знаковые системы': 'Наборщик, кассир, делопроизводитель, бухгалтер, программист, чертежник, корректор, экономист, радист, оператор ПЭВМ, машинистка, наборщик и т.п.',
            'Человек-художественный образ': 'Парикмахер, модельер, чеканщик, маляр, гравер, резчик по камню, фотограф, актер, художник, музыкант и т.п.'}

        global dct_desciprion
        dct_desciprion = \
{'Человек-природа': """Человек-природа.\n
Представителей этих профессий объединяет одно очень важное качество — любовь к 
природе. Но любовь не созерцательная. Которой обладают практически все люди, 
считая природу наиболее благоприятной средой для отдыха, а деятельная связанная с
познанием ее законов и применением их. Одно дело — любить животных и растения,
играть с ними, радоваться им. И совсем другое — регулярно, день за днем ухаживать
за ними, наблюдать, лечить, выгуливать, не считаясь с личным временем и планами.
Специалист должен не просто все знать о живых организмах, но и прогнозировать
возможные изменения в них и принимать меры. От человека требуется инициатива и
самостоятельность в решении конкретных задач, заботливость, терпение и
дальновидность.\n 
Человек, работающий в сфере «человек-природа», должен быть спокойным и уравновешенным.""",
              'Человек-техника': """Человек-техника.\n
Особенность технических объектов в том, что они, как правило, могут быть точно
измерены по многим признакам. При их обработке, преобразовании, перемещении
или оценке от работника требуется точность, определенность действий. Техника как
предмет труда представляет широкие возможности для новаторства, выдумки,
творчества, поэтому важное значение приобретает такое качество, как практическое
мышление. Техническая фантазия, способность мысленно соединять и разъединять
технические объекты и их части — важные условия для успеха в данной области.
""", 'Человек-человек': """Человек-человек.\n
Главное содержание труда в профессиях типа «человек-человек» сводится к
взаимодействию между людьми. Если не наладится это взаимодействие, значит, не
наладится и работа. Качества, необходимые для работы с людьми: устойчивое,
хорошее настроение в процессе работы с людьми, потребность в общении,
способность мысленно ставить себя на место другого человека, быстро понимать
намерения, помыслы, настроение людей, умение разбираться в человеческих
взаимоотношениях, хорошая память (умение держать в уме имена и особенности
многих людей), терпение.
""", 'Человек-знаковые системы': """Человек-знаковая система.\n
Мы встречаемся со знаками значительно чаще, чем обычно представляем себе. Это 
цифры. Коды, условные знаки, естественные или искусственные языки, чертежи,
таблицы формулы. В любом случае человек воспринимает знак как символ реального
объекта или явления. Поэтому специалисту, который работает со знаками, важно
уметь, с одной стороны, абстрагироваться от реальных физических, химически,
механических свойств предметов, а с другой —представлять и воспринимать
характеристики реальных явлений или объектов, стоящих за знаками. Чтобы успешно
работать в какой-нибудь профессии данного типа, необходимо уметь мысленно
погружаться в мир, казалось бы, сухих обозначений и сосредотачиваться на
сведениях, которые они несут в себе. Особые требования профессии этого типа
предъявляют к вниманию.
""", 'Человек-художественный образ': """Человек-художественный образ.\n
Важнейшие требования, которые предъявляют профессии, связанные с изобразительной, музыкальной, литературно-художественной, актерско-сценической деятельностью человека—
Наличие способности к искусствам, творческое воображение, образное мышление, талант, трудолюбие.
"""}

        df = pd.read_excel(file_data_xlsx_ddo)
        threshold_ddo = var_entry_threshold_ddo.get()  # количество колонок не относящихся к вопросам
        if df.shape[1] -threshold_ddo <=20:
            raise WrongNumberColumn

        base_df = df.iloc[:, :threshold_ddo]  # создаем датафрейм с данными не относящимися к тесту
        # делаем строковыми названия колонок
        base_df.columns = list(map(str, base_df.columns))

        # заменяем пробелы на нижнее подчеркивание и очищаем от пробельных символов в начале и конце
        base_df.columns = [column.strip().replace(' ', '_') for column in base_df.columns]

        # очищаем от всех символов кроме букв цифр
        base_df.columns = [re.sub(r'[^_\d\w]', '', column) for column in base_df.columns]

        answers_df = df.iloc[:, threshold_ddo:threshold_ddo+20]  # датафрейм с результатами

        answers_df.columns = [f'ДДО_Вопрос_ №_{i}' for i in range(1, answers_df.shape[1] + 1)]

        answers_df = answers_df.astype(str)

        answers_df = answers_df.applymap(lambda x: x.strip())

        answers_df['ДДО_Необработанный_результат'] = answers_df.apply(processing_result_ddo, axis=1)
        answers_df['ДДО_Необработанный_результат'] = answers_df['ДДО_Необработанный_результат'].astype(str)

        answers_df['ДДО_Обработанный_результат'] = answers_df.apply(processing_finish_result_ddo, axis=1)
        answers_df['ДДО_Обработанный_результат'] = answers_df['ДДО_Обработанный_результат'].astype(str)
        answers_df['ДДО_Описание_результата'] = answers_df['ДДО_Обработанный_результат'].apply(create_out_str_ddo)

        # соединяем после обработки
        df = pd.concat([base_df, answers_df], axis=1)

        # Проверяем наличие колонки ФИО, если она есть то добавляем ее значение, если нет не используем

        if 'ФИО' in df.columns:
            df['ДДО_Описание_результата'] = df['ФИО'] + '.' + ' \nДифференциально-диагностический опросник.\n' + df[
                'ДДО_Описание_результата']
        else:
            df['ДДО_Описание_результата'] = 'Дифференциально-диагностический опросник.\n' + df[
                'ДДО_Описание_результата']
        # генерируем текущее время
        t = time.localtime()
        current_time = time.strftime('%H_%M_%S', t)

        df.to_excel(f'{path_to_end_folder_ddo}/Полная таблица с результатами ДДО от {current_time}.xlsx', index=False,
                    engine='xlsxwriter')

        # Создаем сокращенный вариант
        send_df = df.iloc[:, :threshold_ddo]
        # Добавляем колонки с результатами
        send_df['ДДО_Необработанный_результат'] = df['ДДО_Необработанный_результат']
        send_df['ДДО_Обработанный_результат'] = df['ДДО_Обработанный_результат']
        send_df['ДДО_Описание_результата'] = df['ДДО_Описание_результата']
        send_df.to_excel(f'{path_to_end_folder_ddo}/Краткая таблица с результатами ДДО  от {current_time}.xlsx',
                         index=False, engine='xlsxwriter')











    except NameError:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'Выберите файлы с данными и папку куда будет генерироваться файл')
    except KeyError as e:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'В таблице не найдена указанная колонка {e.args}')
    except FileNotFoundError:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'Перенесите файлы которые вы хотите обработать в корень диска. Проблема может быть\n '
                             f'в слишком длинном пути к обрабатываемым файлам')
    except WrongNumberColumn:
        messagebox.showerror('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                             f'Неправильное количество колонок в таблице!\n'
                             f'Проверьте количество вопросов в тестах!\n'
                             f'ДЦОК -41 колонка т.е.41 тестовый вопрос\n'
                             f'ОПТЛ - 30 колонок т.е. 30 тестовых вопросов\n'
                             f'СППУ - 24 колонки т.е. 24 тестовых вопроса\n'
                             f'ДДО - 20 колонок т.е. 20 тестовых вопросов')
    else:
        messagebox.showinfo('Лахезис Обработка результатов профориентационных тестов ver 1.3',
                            'Данные успешно обработаны')


In [ ]:
params_df = pd.read_excel(file_params,header=None) # считываем файл с параметрами

In [ ]:
params_df

In [33]:
# создаем словарь с параметрами скриптов формата {аббревиатура:(название функции для обработки, количество вопросов(колонок))}
DCT_PARAMS_SCRIPT = {'ДЦОК':(processing_dcok,41),'ОПТЛ':(processing_optl,30),'СППУ':(processing_sppu,24),'ДДО':(processing_ddo,20)}





In [37]:
DCT_PARAMS_SCRIPT['ДЦОК'][0]()

NameError: name 'messagebox' is not defined

In [ ]:
primal_df = pd.read_excel(file_data_xlsx_complex)
# проверяем количество колонок
if primal_df.shape[1] - threshold_complex >= 115:
    raise WrongNumberColumn

base_df = primal_df.iloc[:, :threshold_complex]  # создаем датафрейм с данными не относящимися к тесту
# делаем строковыми названия колонок
base_df.columns = list(map(str, base_df.columns))

# заменяем пробелы на нижнее подчеркивание и очищаем от пробельных символов в начале и конце
base_df.columns = [column.strip().replace(' ', '_') for column in base_df.columns]

# очищаем от всех символов кроме букв цифр
base_df.columns = [re.sub(r'[^_\d\w]', '', column) for column in base_df.columns]

# Создаем копию базового датафрейма для создания отдельных результатов
sep_base_df = base_df.copy()

# Создаем копию базового датафрейма для создания сокращенных общих результатов
short_base_df = base_df.copy()

# Создаем датафрейм ДЦОК диагностика ценностных ориентаций в карьере
dcok_df = primal_df.iloc[:, threshold_complex:threshold_complex + 41]  # датафрейм с результатами

dcok_df.columns = [f'ДЦОК_Вопрос_ №_{i}' for i in range(1, dcok_df.shape[1] + 1)]

dcok_df = dcok_df.astype(int)

# Создаем колонку для результов первичного подсчета
dcok_df['ДЦОК_Необработанный_результат'] = dcok_df.apply(processing_result_dcok, axis=1)
dcok_df['ДЦОК_Необработанный_результат'] = dcok_df['ДЦОК_Необработанный_результат'].astype(str)

# Создаем колонку для итоговой обработки
dcok_df['ДЦОК_Обработанный_результат'] = dcok_df.apply(processing_finish_result_dcok, axis=1)
dcok_df['ДЦОК_Обработанный_результат'] = dcok_df['ДЦОК_Обработанный_результат'].astype(str)

# соединяем после обработки в базовый и в отдельный датафрейм
df = pd.concat([base_df, dcok_df], axis=1)
sep_dcok_df = pd.concat([sep_base_df, dcok_df], axis=1)

dcok_description_result = """
По каждой из девяти карьерных ориентаций подсчитывается количество баллов. Таким образом определяется ведущая карьерная ориентация - количество набранных для этого баллов должно быть не менее пяти. Иногда ведущей не становится ни одна карьерная ориентация - в таком случае карьера не является центральной в жизни личности. 

1) Профессиональная компетентность -быть профессионалом, мастером в своем деле. 
Эта ориентация связана с наличием способностей и талантов в определенной области.
Люди с такой ориентацией хотят быть мастерами своего дела, они бывают особенно счастливы, когда достигают успеха в профессиональной сфере, но быстро теряют интерес к работе, которая не позволяет развивать их способности. Вряд ли их заинтересует даже значительно более высокая должность, если она не связана с их профессиональными компетенциями. Они ищут признания своих талантов, что должно выражаться в статусе, соответствующем их мастерству. Они готовы управлять другими в пределах своей компетенции, но управление не представляет для них особого интереса. Поэтому многие из этой категории отвергают работу руководителя, управление рассматривают как необходимое условие для продвижения в своей профессиональной сфере.
2) Менеджмент - Управлять – людьми, проектами, бизнес-процессами и т.п.
Для этих людей первостепенное значение имеет ориентация личности на интеграцию усилий других людей, полнота ответственности за конечный результат и соединение различных функций организации. С возрастом и опытом эта карьерная ориентация проявляется сильнее. Возможности для лидерства, высокого дохода, повышенных уровней ответственности и вклад в успех своей организации являются ключевыми ценностями и мотивами. Самое главное для них – управление: людьми, проектами, любыми бизнес-процессами – это в целом не имеет принципиального значения. Центральное понятие их профессионального развития – власть, осознание того, что от них зависит принятие ключевых решений. Причем для них не является принципиальным управление собственным проектом или целым бизнесом, скорее наоборот, они в большей степени ориентированы на построение карьеры в наемном менеджменте, но при условии, что им будут делегированы значительные полномочия. Человек с такой ориентацией будет считать, что не достиг цели своей карьеры, пока не займет должность, на которой будет управлять различными сторонами деятельности предприятия.
3) Автономия (независимость) – Главное в работе – это свобода и независимость. 
Первичная забота личности с такой ориентацией –освобождение от организационных правил, предписаний и ограничений. Они испытывают трудности, связанные с установленными правилами, процедурами, рабочим днем, дисциплиной, формой одежды и т.д. Они любят выполнять работу своим способом, темпом и по собственным стандартам. Они не любят, когда работа вмешивается в их частную жизнь, поэтому предпочитают делать независимую карьеру собственным путем. Они скорее выберут низкосортную работу, чем откажутся от автономии и независимости. Для них первоочередная задача развития карьеры – получить возможность работать самостоятельно, самому решать, как, когда и что делать для достижения тех или иных целей. Карьера для них – это, прежде всего, способ реализации их свободы, поэтому любые рамки и строгое подчинение оттолкнут их даже от внешне привлекательной вакансии. Такой человек может работать в организации, которая обеспечивает достаточную степень свободы.
4) Стабильность работы - стабильная, надежная работа на длительное время.
Эти люди испытывают потребность в безопасности, защите и возможности прогнозирования и будут искать постоянную работу с минимальной вероятностью увольнения. Эти люди отождествляют свою работу со своей карьерой. Их потребность в безопасности и стабильности ограничивает выбор вариантов	карьеры.
Авантюрные или краткосрочные проекты и только становящиеся на ноги компании их, скорее всего, не привлекают. Они очень ценят социальные гарантии, которые может предложить работодатель, и, как правило, их выбор места работы связан именно с длительным контрактом и стабильным положением компании на рынке. Такие люди ответственность за управление своей карьерой перекладывают на нанимателя. Часто данная ценностная ориентация сочетается с невысоким уровнем притязаний.
5) Стабильность места жительства - Главное – жить в своем городе (минимум переездов, командировок).
Важнее остаться на одном месте жительства, чем получить повышение или новую работу на новой местности. Переезд для таких людей неприемлем, и даже частые командировки являются для них негативным фактором при рассмотрении	предложения о работе.
6) Служение - Воплощать в работе свои идеалы и ценности.
Данная ценностная ориентация характерна для людей, занимающихся делом по причине желания реализовать в своей работе главные ценности. Они часто ориентированы больше на ценности, чем на требующиеся в данном виде работы способности. Они стремятся приносить пользу людям, обществу, для них очень важно видеть конкретные плоды своей работы, даже если они и не выражены в материальном эквиваленте. Основной тезис построения их карьеры – получить возможность максимально эффективно использовать их таланты и опыт для реализации общественно	важной цели. Люди, ориентированные на служение, общительны и часто консервативны. Человек с такой ориентацией не будет работать в организации, которая враждебна его целям и ценностям.
7) Вызов - Сделать   невозможное – возможным, решать   уникальные   задачи. 
Эти люди считают успехом преодоление непреодолимых препятствий, решение неразрешимых проблем или просто выигрыш. Они ориентированы на то, чтобы “бросать вызов”. Для одних людей вызов представляет более трудная работа, для других это — конкуренция и межличностные отношения. Они ориентированы на решение заведомо сложных задач, преодоление препятствий ради победы в конкурентной борьбе. Они чувствуют себя преуспевающими только тогда, когда постоянно вовлечены в решение трудных проблем или в ситуацию соревнования. Карьера для них – это постоянный вызов их профессионализму, и они всегда готовы его принять. Социальная ситуация чаще всего рассматривается с позиции “выигрыша – проигрыша”. Процесс борьбы и победа более важна для них, чем конкретная область деятельности или квалификация. Новизна, разнообразие и вызов имеют для них очень большую ценность, и, если все идет слишком просто, им становиться скучно.
8) Интеграция стилей жизни - Сохранение гармонии между сложившейся личной жизнью и карьерой.
Для людей этой категории карьера должна ассоциироваться с общим стилем жизни, уравновешивая потребности человека, семьи и карьеры. Они хотят, чтобы организационные отношения отражали бы уважение к их личным и семейным проблемам.
Выбирать и поддерживать определенный образ жизни для них важнее, чем добиваться успеха в карьере. Развитие карьеры их привлекает только в том случае, если она не нарушает привычный им стиль жизни и окружение. Для них важно, чтобы все было уравновешено – карьера, семья, личные интересы и т.п. Жертвовать   чем-то   одним   ради   другого   им    явно    не    свойственно. Такие люди обычно в своем поведении проявляют конформность (тенденция изменять свое поведение в зависимости от влияния других людей с тем, чтобы оно соответствовало мнению окружающих)
9) Предпринимательство – Создавать новые организации, товары, услуги.
Этим людям нравится создавать новые организации, товары или услуги, которые могут быть отождествлены с их усилиями. Работать на других – это не их, они – предприниматели по духу, и цель их карьеры – создать что-то новое, организовать свое дело, воплотить в жизнь идею, всецело принадлежащую только им. Вершина карьеры в их понимании – собственный бизнес.
"""

# Обрабатываем базовый датафрейм
if 'ФИО' in df.columns:
    df['ДЦОК_Описание_результата'] = df['ФИО'] +'.'+ ' \nДиагностика ценностных ориентаций в карьере.\n' + df[
        'ДЦОК_Обработанный_результат'] + dcok_description_result
else:
    df['ДЦОК_Описание_результата'] = 'Диагностика ценностных ориентаций в карьере.\n' + df[
        'ДЦОК_Обработанный_результат'] + dcok_description_result

# Обрабатываем ДЦОК датафрейм

if 'ФИО' in sep_dcok_df.columns:
    sep_dcok_df['ДЦОК_Описание_результата'] = sep_dcok_df['ФИО'] +'.'+ ' \nДиагностика ценностных ориентаций в карьере.\n' + \
                                              sep_dcok_df['ДЦОК_Обработанный_результат'] + dcok_description_result
else:
    sep_dcok_df['ДЦОК_Описание_результата'] = 'Диагностика ценностных ориентаций в карьере.\n' + sep_dcok_df[
        'ДЦОК_Обработанный_результат'] + dcok_description_result

# Добавляем в базовый сокращенный датафрейм
short_base_df['ДЦОК_Необработанный_результат'] = df['ДЦОК_Необработанный_результат']
short_base_df['ДЦОК_Обработанный_результат'] = df['ДЦОК_Обработанный_результат']
short_base_df['ДЦОК_Описание_результата'] = df['ДЦОК_Описание_результата']

# Сохраняем ДЦОК датафрейм
# генерируем текущее время
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)

sep_dcok_df.to_excel(f'{path_to_end_folder_complex}/Полная таблица с результатами ДЦОК от {current_time}.xlsx',
                     index=False, engine='xlsxwriter')

# Создаем сокращенный вариант
send_sep_dcok_df = sep_dcok_df.iloc[:, :threshold_complex]
# Добавляем колонки с результатами
send_sep_dcok_df['ДЦОК_Обработанный_результат'] = sep_dcok_df['ДЦОК_Обработанный_результат']
send_sep_dcok_df['ДЦОК_Описание_результата'] = sep_dcok_df['ДЦОК_Описание_результата']
send_sep_dcok_df.to_excel(
    f'{path_to_end_folder_complex}/Краткая таблица с результатами ДЦОК  от {current_time}.xlsx', index=False,
    engine='xlsxwriter')

"""
    Обработка результатов ОПТЛ
    """

# Создаем датафрейм ОПТЛ диагностика ценностных ориентаций в карьере
optl_df = primal_df.iloc[:, threshold_complex + 41:threshold_complex + 71]  # датафрейм с результатами

optl_df.columns = [f'ОПТЛ_Вопрос_ №_{i}' for i in range(1, optl_df.shape[1] + 1)]

optl_df = optl_df.astype(str)
optl_df = optl_df.applymap(lambda x: x.strip())

# Создаем колонку для результов первичного подсчета
optl_df['ОПТЛ_Обработанный_результат'] = optl_df.apply(processing_result_optl, axis=1)
optl_df['ОПТЛ_Обработанный_результат'] = optl_df['ОПТЛ_Обработанный_результат'].astype(str)

# соединяем после обработки в базовый и в отдельный датафрейм
df = pd.concat([df, optl_df], axis=1)
sep_optl_df = pd.concat([sep_base_df, optl_df], axis=1)

optl_description_result = """
Обработка результатов теста 
8-10 баллов – ярко выраженный тип;  
5-7 баллов – средне выраженный тип;  
2-4 баллов – слабо выраженный тип.  
Наибольшее количество баллов указывает на доминирующий тип. В чистом виде эти профессиональные типы встречаются редко – обычно можно говорить только о преобладающем типе личности. Выбирая профессию, необходимо учитывать свой профессиональный тип. Если профессия не соответствует вашему типу личности, работа будет даваться вам ценой значительного нервно-психического напряжения.  
Интерпретация результатов теста 
1. Реалистический тип (Р) 
Люди, относящиеся к этому типу, предпочитают выполнять работу, требующую силы, ловкости, подвижности, хорошей координации движений, навыков практической работы. Результаты труда профессионалов этого типа ощутимы и реальны – их руками создан весь окружающий нас предметный мир. Люди реалистического типа охотнее делают, чем говорят, они настойчивы и уверены в себе, в работе предпочитают четкие и конкретные указания. Придерживаются традиционных ценностей, поэтому критически относятся к новым идеям.  
Близкие типы: интеллектуальный и офисный.  
Противоположный тип: социальный.  
Хороший продавец и хороший ремонтник никогда не будут голодать. Шенк 
2. Интеллектуальный (И) 
Людей, относящихся к этому типу, отличают аналитические способности, рационализм, независимость и оригинальность мышления, умение точно формулировать и излагать свои мысли, решать логические задачи, генерировать новые идеи. Они часто выбирают научную и исследовательскую работу. Им нужна свобода для творчества. Работа способна увлечь их настолько, что стирается грань между рабочим временем и досугом. Мир идей для них может быть важнее, чем общение с людьми. Материальное благополучие для них обычно не на первом месте.  
Близкие типы: реалистический и артистический.  
Противоположный тип: предпринимательский.  
Научная работа не подходит человеку, который обеими ногами стоит на земле и обеими руками тянется к долларам. М.Ларни 
3. Социальный (С) 
Люди, относящиеся к этому типу, предпочитают профессиональную деятельность, связанную с обучением, воспитанием, лечением, консультированием, обслуживанием. Люди этого типа гуманны, чувствительны, активны, ориентированы на социальные нормы, способны понять эмоциональное состояние другого человека. Для них характерно хорошее речевое развитие, живая мимика, интерес к людям, готовность прийти на помощь. Материальное благополучие для них обычно не на первом месте.  
Близкие типы: артистический и предпринимательский.  
Противоположный тип: реалистический.  
Если больному после разговора с врачом не стало легче, то это не врач. В.Бехтерев 
4. Офисный (О) 
Люди этого типа обычно проявляют склонность к работе, связанной с обработкой и систематизацией информации, предоставленной в виде условных знаков, цифр, формул, текстов (ведение документации, установление количественных соотношений между числами и условными знаками). Они отличаются аккуратностью, пунктуальностью, практичностью, ориентированы на социальные нормы, предпочитают четко регламентированную работу. Материальное благополучие для них более значимо, чем для других типов. Склонны к работе, не связанной с широкими контактами и принятием ответственных решений.  
Близкие типы: реалистический и предпринимательский.  
Противоположный тип: артистический.  
Офис может работать без шефа, но не без секретаря. Дж.Фонда 
5. Предпринимательский (П)  
Люди этого типа находчивы, практичны, быстро ориентируются в сложной обстановке, склонны к самостоятельному принятию решений, социально активны, готовы рисковать, ищут острые ощущения. Любят и умеют общаться. Имеют высокий уровень притязаний. Избегают занятий, требующих усидчивости, большой и длительной концентрации внимания. Для них значимо материальное благополучие. Предпочитают деятельность, требующую энергии, организаторских способностей, связанную с руководством, управлением и влиянием на людей.  
Близкие типы: офисный и социальный.  
Противоположный тип: исследовательский.  
Специальность налетчика куда менее заманчива, чем смежные с ней профессии политика или биржевого спекулянта. О.Генри  
6. Артистический (А)  
Люди этого типа оригинальны, независимы в принятии решений, редко ориентируются на социальные нормы и одобрение, обладают необычным взглядом на жизнь, гибкостью мышления, эмоциональной чувствительностью. Отношения с людьми строят, опираясь на свои ощущения, эмоции, воображение, интуицию. Они не выносят жесткой регламентации, предпочитая свободный график работы. Часто выбирают профессии, связанные с литературой, театром, кино, музыкой, изобразительным искусством.  
Близкие типы: интеллектуальный и социальный.  
Противоположный тип: офисный.  
Только поэты и женщины умеют обращаться с деньгами так, как деньги того заслуживают. А.Боннар 
"""

# Обрабатываем базовый датафрейм
if 'ФИО' in df.columns:
    df['ОПТЛ_Описание_результата'] = df['ФИО'] +'.'+ ' \nОпределение профессионального типа личности.\n' + df[
        'ОПТЛ_Обработанный_результат'] + optl_description_result
else:
    df['ОПТЛ_Описание_результата'] = 'Определение профессионального типа личности.\n' + df[
        'ОПТЛ_Обработанный_результат'] + optl_description_result

# Обрабатываем ОПТЛ датафрейм

if 'ФИО' in sep_optl_df.columns:
    sep_optl_df['ОПТЛ_Описание_результата'] = sep_optl_df['ФИО'] +'.'+ ' \nОпределение профессионального типа личности.\n' + \
                                              sep_optl_df['ОПТЛ_Обработанный_результат'] + optl_description_result
else:
    sep_optl_df['ОПТЛ_Описание_результата'] = 'Определение профессионального типа личности.\n' + sep_optl_df[
        'ОПТЛ_Обработанный_результат'] + optl_description_result

# Добавляем в базовый сокращенный датафрейм
short_base_df['ОПТЛ_Обработанный_результат'] = df['ОПТЛ_Обработанный_результат']
short_base_df['ОПТЛ_Описание_результата'] = df['ОПТЛ_Описание_результата']

# Сохраняем ОПТЛ датафрейм
# генерируем текущее время
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)

sep_optl_df.to_excel(f'{path_to_end_folder_complex}/Полная таблица с результатами ОПТЛ от {current_time}.xlsx',
                     index=False, engine='xlsxwriter')

# Создаем сокращенный вариант
send_sep_optl_df = sep_optl_df.iloc[:, :threshold_complex]
# Добавляем колонки с результатами
send_sep_optl_df['ОПТЛ_Обработанный_результат'] = sep_optl_df['ОПТЛ_Обработанный_результат']
send_sep_optl_df['ОПТЛ_Описание_результата'] = sep_optl_df['ОПТЛ_Описание_результата']
send_sep_optl_df.to_excel(
    f'{path_to_end_folder_complex}/Краткая таблица с результатами ОПТЛ  от {current_time}.xlsx', index=False,
    engine='xlsxwriter')

"""
    Обработка результатов СППУ Сфера профессиональных предпочтений учащихся
    """

# Создаем датафрейм СППУ диагностика ценностных ориентаций в карьере
sppu_df = primal_df.iloc[:, threshold_complex + 71:threshold_complex + 95]  # датафрейм с результатами

sppu_df.columns = [f'СППУ_Вопрос_ №_{i}' for i in range(1, sppu_df.shape[1] + 1)]

sppu_df = sppu_df.astype(str)

sppu_df = sppu_df.applymap(lambda x: x.strip())

# Создаем колонку для результов первичного подсчета
sppu_df['СППУ_Обработанный_результат'] = sppu_df.apply(processing_result_sppu, axis=1)
sppu_df['СППУ_Обработанный_результат'] = sppu_df['СППУ_Обработанный_результат'].astype(str)

# соединяем после обработки в базовый и в отдельный датафрейм
df = pd.concat([df, sppu_df], axis=1)
sep_sppu_df = pd.concat([sep_base_df, sppu_df], axis=1)

sppu_description_result = """
Шкала оценки результатов
10-12 баллов – ярко выраженная профессиональная склонность;
7-9 баллов – средне выраженная профессиональная склонность;
4-6 баллов – слабо выраженная профессиональная склонность;
0-3 баллов – профессиональная склонность не выражена.

Примеры профессий для каждой сферы
Сфера работы с людьми: учитель, педагог, экскурсовод, воспитатель,  социолог, психолог, менеджер по персоналу, следователь.
Сфера умственного труда: ученый-исследователь (математик, физик, химик, кибернетик, археолог, геолог),  инженер, юрист, врач, эколог, архитектор, продюсер.
Сфера технических интересов:  программист, электротехник, радиотехник, Web-мастер, статистик, водитель, технолог, диспетчер, секретарь-машинистка, телефонист.
Сфера эстетики и искусства: художник, дизайнер, писатель, поэт,  режиссер, артист, конструктор, косметолог, костюмер, гример, кондитер, портной-кутюрье, цветовод. 
Сфера физического труда,  подвижной деятельности: спортсмен, фотограф, экспедитор, парикмахер,  бармен, официант, стюардесса, продавец, закройщик, специалист по ремонту, кассир, медперсонал, бригадир, кладовщик,  почтальон,  фермер,  водитель-дальнобойщик, полицейский, военный.
Сфера материальных интересов,  планово-экономических видов работ: экономист, администратор, менеджер, предприниматель, аудитор, специалист по рекламе, брокер, агент страховых компаний, коммерсант, завхоз.
"""

# Обрабатываем базовый датафрейм
if 'ФИО' in df.columns:
    df['СППУ_Описание_результата'] = df['ФИО'] +'.'+ ' \nСфера профессиональных предпочтений учащихся.\n' + df[
        'СППУ_Обработанный_результат'] + sppu_description_result
else:
    df['СППУ_Описание_результата'] = 'Сфера профессиональных предпочтений учащихся.\n' + df[
        'СППУ_Обработанный_результат'] + sppu_description_result

# Обрабатываем СППУ датафрейм

if 'ФИО' in sep_sppu_df.columns:
    sep_sppu_df['СППУ_Описание_результата'] = sep_sppu_df['ФИО'] +'.'+ ' \nСфера профессиональных предпочтений учащихся.\n' + \
                                              sep_sppu_df['СППУ_Обработанный_результат'] + sppu_description_result
else:
    sep_sppu_df['СППУ_Описание_результата'] = 'Сфера профессиональных предпочтений учащихся.\n' + sep_sppu_df[
        'СППУ_Обработанный_результат'] + sppu_description_result

# Добавляем в базовый сокращенный датафрейм
short_base_df['СППУ_Обработанный_результат'] = df['СППУ_Обработанный_результат']
short_base_df['СППУ_Описание_результата'] = df['СППУ_Описание_результата']

# Сохраняем СППУ датафрейм
# генерируем текущее время
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)

sep_sppu_df.to_excel(f'{path_to_end_folder_complex}/Полная таблица с результатами СППУ от {current_time}.xlsx',
                     index=False, engine='xlsxwriter')

# Создаем сокращенный вариант
send_sep_sppu_df = sep_sppu_df.iloc[:, :threshold_complex]
# Добавляем колонки с результатами
send_sep_sppu_df['СППУ_Обработанный_результат'] = sep_sppu_df['СППУ_Обработанный_результат']
send_sep_sppu_df['СППУ_Описание_результата'] = sep_sppu_df['СППУ_Описание_результата']
send_sep_sppu_df.to_excel(
    f'{path_to_end_folder_complex}/Краткая таблица с результатами СППУ  от {current_time}.xlsx', index=False,
    engine='xlsxwriter')

"""
    Обработка результатов ДДО Дифференциально-диагностический опросник
    """

# Создаем датафрейм ДДО диагностика ценностных ориентаций в карьере
ddo_df = primal_df.iloc[:, threshold_complex + 95:threshold_complex + 115]  # датафрейм с результатами

ddo_df.columns = [f'ДДО_Вопрос_ №_{i}' for i in range(1, ddo_df.shape[1] + 1)]

ddo_df = ddo_df.astype(str)

ddo_df = ddo_df.applymap(lambda x: x.strip())

# Создаем словари для создания текста письма
global dct_prof
dct_prof = {
    'Человек-природа': 'Тракторист, рыбовод, зоотехник, агроном, садовник, ветеринар, животновод, геолог, биолог, почвовед и т.д.',
    'Человек-техника': 'Водитель, токарь, инженер, слесарь, радиотехник, швея, электрик, механик, монтажник и т.п.',
    'Человек-человек': 'Продавец, медсестра, секретарь, бортпроводник, учитель, воспитатель, няня, преподаватель, врач, официант, администратор и т.п.',
    'Человек-знаковые системы': 'Наборщик, кассир, делопроизводитель, бухгалтер, программист, чертежник, корректор, экономист, радист, оператор ПЭВМ, машинистка, наборщик и т.п.',
    'Человек-художественный образ': 'Парикмахер, модельер, чеканщик, маляр, гравер, резчик по камню, фотограф, актер, художник, музыкант и т.п.'}

global dct_desciprion
dct_desciprion =\
{'Человек-природа': """Человек-природа.\n
Представителей этих профессий объединяет одно очень важное качество — любовь к 
природе. Но любовь не созерцательная. Которой обладают практически все люди, 
считая природу наиболее благоприятной средой для отдыха, а деятельная связанная с
познанием ее законов и применением их. Одно дело — любить животных и растения,
играть с ними, радоваться им. И совсем другое — регулярно, день за днем ухаживать
за ними, наблюдать, лечить, выгуливать, не считаясь с личным временем и планами.
Специалист должен не просто все знать о живых организмах, но и прогнозировать
возможные изменения в них и принимать меры. От человека требуется инициатива и
самостоятельность в решении конкретных задач, заботливость, терпение и
дальновидность.\n 
Человек, работающий в сфере «человек-природа», должен быть спокойным и уравновешенным.""",
'Человек-техника': """Человек-техника.\n
Особенность технических объектов в том, что они, как правило, могут быть точно
измерены по многим признакам. При их обработке, преобразовании, перемещении
или оценке от работника требуется точность, определенность действий. Техника как
предмет труда представляет широкие возможности для новаторства, выдумки,
творчества, поэтому важное значение приобретает такое качество, как практическое
мышление. Техническая фантазия, способность мысленно соединять и разъединять
технические объекты и их части — важные условия для успеха в данной области.
""", 'Человек-человек': """Человек-человек.\n
Главное содержание труда в профессиях типа «человек-человек» сводится к
взаимодействию между людьми. Если не наладится это взаимодействие, значит, не
наладится и работа. Качества, необходимые для работы с людьми: устойчивое,
хорошее настроение в процессе работы с людьми, потребность в общении,
способность мысленно ставить себя на место другого человека, быстро понимать
намерения, помыслы, настроение людей, умение разбираться в человеческих
взаимоотношениях, хорошая память (умение держать в уме имена и особенности
многих людей), терпение.
""", 'Человек-знаковые системы': """Человек-знаковая система.\n
Мы встречаемся со знаками значительно чаще, чем обычно представляем себе. Это 
цифры. Коды, условные знаки, естественные или искусственные языки, чертежи,
таблицы формулы. В любом случае человек воспринимает знак как символ реального
объекта или явления. Поэтому специалисту, который работает со знаками, важно
уметь, с одной стороны, абстрагироваться от реальных физических, химически,
механических свойств предметов, а с другой —представлять и воспринимать
характеристики реальных явлений или объектов, стоящих за знаками. Чтобы успешно
работать в какой-нибудь профессии данного типа, необходимо уметь мысленно
погружаться в мир, казалось бы, сухих обозначений и сосредотачиваться на
сведениях, которые они несут в себе. Особые требования профессии этого типа
предъявляют к вниманию.
""", 'Человек-художественный образ': """Человек-художественный образ.\n
Важнейшие требования, которые предъявляют профессии, связанные с изобразительной, музыкальной, литературно-художественной, актерско-сценической деятельностью человека—
Наличие способности к искусствам, творческое воображение, образное мышление, талант, трудолюбие.
"""}

# Создаем колонку для результов первичного подсчета
ddo_df['ДДО_Необработанный_результат'] = ddo_df.apply(processing_result_ddo, axis=1)
ddo_df['ДДО_Необработанный_результат'] = ddo_df['ДДО_Необработанный_результат'].astype(str)

ddo_df['ДДО_Обработанный_результат'] = ddo_df.apply(processing_finish_result_ddo, axis=1)
ddo_df['ДДО_Обработанный_результат'] = ddo_df['ДДО_Обработанный_результат'].astype(str)

ddo_df['ДДО_Описание_результата'] = ddo_df['ДДО_Обработанный_результат'].apply(create_out_str_ddo)

# соединяем после обработки в базовый и в отдельный датафрейм
df = pd.concat([df, ddo_df], axis=1)
sep_ddo_df = pd.concat([sep_base_df, ddo_df], axis=1)

# Обрабатываем базовый датафрейм
if 'ФИО' in df.columns:
    df['ДДО_Описание_результата'] = df['ФИО'] +'.'+ ' \nДифференциально-диагностический опросник.\n' + df[
        'ДДО_Описание_результата']
else:
    df['ДДО_Описание_результата'] = 'Дифференциально-диагностический опросник.\n' + df['ДДО_Описание_результата']

# Обрабатываем ДДО датафрейм

if 'ФИО' in sep_ddo_df.columns:
    sep_ddo_df['ДДО_Описание_результата'] = sep_ddo_df['ФИО'] +'.'+ ' \nДифференциально-диагностический опросник.\n' + \
                                            sep_ddo_df['ДДО_Описание_результата']
else:
    sep_ddo_df['ДДО_Описание_результата'] = 'Дифференциально-диагностический опросник.\n' + sep_ddo_df[
        'ДДО_Описание_результата']

# Добавляем в базовый сокращенный датафрейм
short_base_df['ДДО_Необработанный_результат'] = df['ДДО_Необработанный_результат']
short_base_df['ДДО_Обработанный_результат'] = df['ДДО_Обработанный_результат']
short_base_df['ДДО_Описание_результата'] = df['ДДО_Описание_результата']

# Сохраняем ДДО датафрейм
# генерируем текущее время
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)

sep_ddo_df.to_excel(f'{path_to_end_folder_complex}/Полная таблица с результатами ДДО от {current_time}.xlsx',
                    index=False, engine='xlsxwriter')
# Создаем сокращенный вариант
send_sep_ddo_df = sep_ddo_df.iloc[:, :threshold_complex]
# Добавляем колонки с результатами
send_sep_ddo_df['ДДО_Необработанный_результат'] = sep_ddo_df['ДДО_Необработанный_результат']
send_sep_ddo_df['ДДО_Обработанный_результат'] = sep_ddo_df['ДДО_Обработанный_результат']
send_sep_ddo_df['ДДО_Описание_результата'] = sep_ddo_df['ДДО_Описание_результата']

send_sep_ddo_df.to_excel(
    f'{path_to_end_folder_complex}/Краткая таблица с результатами ДДО  от {current_time}.xlsx',
    index=False, engine='xlsxwriter')

df.to_excel(
    f'{path_to_end_folder_complex}/Полная таблица с результатами комплексного тестирования от {current_time}.xlsx',
    index=False, engine='xlsxwriter')
short_base_df.to_excel(
    f'{path_to_end_folder_complex}/Краткая таблица с результатами комплексного тестирования от {current_time}.xlsx',
    index=False, engine='xlsxwriter')
